In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from requests.auth import HTTPBasicAuth

In [37]:
class UserReviewParser:
    PRONS_STR = "переваги"
    СONS_STR = "недоліки"

    MARK_PREFIX = "&q;mark&q;:"

    def __init__(self, url: str):
        self.__url: str = url
        self.__soup = self.__get_soup(url)
        self.__script_text: str = UserReviewParser.__get_script_parser(self.__soup)
        # print(self.__script_text)

    @staticmethod
    def __get_soup(url):
        result = requests.get(url)
        soup = BeautifulSoup(result.text, 'html.parser')
        # print(result.text)
        return soup

    @staticmethod
    def __get_script_parser(soup):
        return soup.find('script', id="rz-client-state").text

    def find_user_mark(self, user_name: str):
        user_pos = self.__script_text.find(user_name)
        if user_pos == -1:
            return None

        mark_prefix_pos = self.__script_text.find(self.MARK_PREFIX, user_pos)
        if mark_prefix_pos == -1:
            return None

        mark_str = self.__script_text[mark_prefix_pos + len(self.MARK_PREFIX)]
        return int(mark_str)

    def find_user_mark_by_title(self, title: str):
        title_pos = self.__script_text.find(title)
        if title_pos == -1:
            return None

        mark_prefix_pos = self.__script_text.find(self.MARK_PREFIX, title_pos - 100)
        if mark_prefix_pos == -1:
            return None

        if mark_prefix_pos > title_pos:
            return None

        mark_str = self.__script_text[mark_prefix_pos + len(self.MARK_PREFIX)]
        try:
            return int(mark_str)
        except:
            return None

    def parse_review_data(self):
        comment_soups = self.__soup.findAll('div', class_='product-comment__body')
        result = {
            'title': [],
            'prons': [],
            'cons': [],
            'mark': []
        }
        for comment_soup in comment_soups:
            try:
                title = comment_soup.p.text
                if self.__detect_language(title) != 'uk':
                    continue

                mark = self.find_user_mark_by_title(title)
                if mark is None or mark == 0:
                    continue
                prons, cons = self.__parse_prons_cons(comment_soup)
                if self.__detect_language(prons) != 'uk':
                    continue

                if self.__detect_language(cons) != 'uk':
                    continue

                result['title'].append(title)
                result['prons'].append(prons)
                result['cons'].append(cons)
                result['mark'].append(mark)

            except:
                print(self.__url)
                continue

        return result

    def __detect_language(self, text: str):
        return 'uk'

    def __parse_prons_cons(self, comment_soup):
        prons, cons = None, None
        essential_soups = comment_soup.findAll('dl', class_='product-comment__essentials')
        for essential_soup in essential_soups:
            dt = essential_soup.find('dt')
            if dt is None:
                continue

            dd = essential_soup.find('dd')
            if dd is None:
                continue

            if self.PRONS_STR in dt.text.lower():
                prons = dd.text

            if self.СONS_STR in dt.text.lower():
                cons = dd.text

        return prons, cons

        

In [23]:
url2 ="https://rozetka.com.ua/ua/asus_90nr01l3_m10560/p182567351/comments/"

In [25]:
result = requests.get(url2)
soup = BeautifulSoup(result.text, 'html.parser')

In [26]:
soup.findAll('div' ,class_='product-comment__body')

[<div class="product-comment__body"><!-- --><p>Подходит ли даная оперативка сюда?<br/>
 <br/>
 Оперативная память HyperX SODIMM DDR4-2666 8192MB PC4-21300 Impact (HX426S15IB2/8) <br/>
  Подробнее: http://hard.rozetka.com.ua/hyperx_hx426s15ib2_8/p45133968/</p><!-- --><!-- --></div>,
 <div class="product-comment__body"><!-- --><p>да</p></div>,
 <div class="product-comment__body"><!-- --><p>Какая тут оперативка?</p><!-- --><!-- --></div>,
 <div class="product-comment__body"><!-- --><p>Подскажите пожалуйста, кто производитель видеокарты которая в ноутбке, msi, asus? Или другая? Спасибо</p><!-- --><!-- --></div>,
 <div class="product-comment__body"><!-- --><p>В ноутбуках немного все иначе. В ПК версиях под производителем подразумевается вендор который ставит свою систему охлаждения. В данном случае можно считать что Асус, так как охлад их и ноут их</p></div>,
 <div class="product-comment__body"><!-- --><p>И да ноут хорош</p></div>,
 <div class="product-comment__body"><!-- --><p>Взял в начал

In [24]:
parser = UserReviewParser(url2)

In [178]:
parser.find_user_mark_by_title('Ко мне часто обращаются с просьбой усовершенствовать подобные ноутбуке. Ситуация, когда человек покупает такой аппарат для работы, думая, что дешевый ноут потянет работу в интернете и офис. Но со временем приходят обновления на систему, google chrome - 1гб оперативной памяти, скайп - ещё 1гб, офис и другие рабочие программы - вот и вся оперативка съедена.')

In [179]:
r = parser.parse_review_data()

In [180]:
pd.DataFrame(r)

,title,prons,cons,mark
0,Тут type C thunderbolt или обычный ?,None,None,4
1,Как оформить кредит на этот ноут,None,None,5
2,Через полтора месяца накрылся кулер ...,"Подсветка, начинка, ips","Кулер слабый, сильно греется; интернет разъем ...",3


In [80]:
parser.find_user_mark('Семён Разжаловец')

3

In [73]:
parser.__script_text

AttributeError: 'UserReviewParser' object has no attribute '__script_text'

In [147]:
soup.find('script', id="rz-client-state")

<script id="rz-client-state" type="application/json">{&q;G.https://product-api.rozetka.com.ua/v3/goods/get-main?front-type=xl&a;id=181862304&a;is_canonical=0&a;lang=ru?&q;:{&q;body&q;:{&q;data&q;:{&q;id&q;:181862304,&q;title&q;:&q;Dell Inspiron 3582 (I35C445DIL-73B) Black&q;,&q;price&q;:&q;4999&q;,&q;old_price&q;:&q;5499&q;,&q;price_pcs&q;:&q;178.54&q;,&q;href&q;:&q;http://rozetka.com.ua/dell_i35c445dil_73b/p181862304/&q;,&q;name&q;:&q;dell_i35c445dil_73b&q;,&q;comments_amount&q;:9,&q;sell_status&q;:&q;available&q;,&q;status&q;:&q;active&q;,&q;status_inherited&q;:&q;active&q;,&q;comments_mark&q;:&q;3.5&q;,&q;category_id&q;:80004,&q;seller_id&q;:5,&q;merchant_id&q;:1,&q;group_id&q;:0,&q;producer_id&q;:7,&q;docket&q;:&q;Экран 15.6\&q; TN+film (1366x768) WXGA HD, глянцевый с антибликовым покрытием / Intel Celeron N4000 (1.1 - 2.6 ГГц) / RAM 4 ГБ / HDD 500 ГБ / Intel UHD 600 / DVD+/-RW / Wi-Fi / Bluetooth / веб-камера / Linux / 2.28 кг / черный&q;,&q;state&q;:&q;new&q;,&q;title_short&q;:&q

In [ ]:
def parse_mark()

In [148]:
pos = soup.find('script', id="rz-client-state").text.find('А сталкер чистое небо потянет хоть на минималка?')
pos

77829

In [149]:
len('&q;mark&q;:5,&q;percent_dignity&q;:&q;0&q;,&q;text&q;:&q;')

57

In [150]:
soup.find('script', id="rz-client-state").text[pos-100: pos + 200]

'5801187,&q;usertitle&q;:&q;Меня Зовут&q;,&q;mark&q;:0,&q;percent_dignity&q;:&q;100&q;,&q;text&q;:&q;А сталкер чистое небо потянет хоть на минималка?&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;26 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;mon'

In [122]:
soup.find('script', id="rz-client-state").text.rfind('mark&q;:', pos)

91607

In [41]:
soup.find('script', id="rz-client-state").text[74770 + len('mark&q;:')]

'3'

In [84]:
soup.findAll('div' ,class_='product-comment__body')

[<div class="product-comment__body"><!-- --><p>А он на операцоной систем віндеус?</p><!-- --><!-- --></div>,
 <div class="product-comment__body"><!-- --><p>Нет. Винду надо самому ставить</p></div>,
 <div class="product-comment__body"><!-- --><p>Добрий варіант на цю ціну.</p><!-- --><dl class="product-comment__essentials"><dt class="product-comment__essentials-label"> Достоинства: </dt><dd>Легкий, тоненький, елегантний. Достатньо шкидкий  - для інтернет серфінгу, скайпу, офісної роботи, фільмів та он-лайн ТБ</dd></dl><!-- --><dl class="product-comment__essentials"><dt class="product-comment__essentials-label"> Недостатки: </dt><dd>Немає індикації роботи диска, зарядки окремими  led діодами</dd></dl></div>,
 <div class="product-comment__body"><!-- --><p>Скільки думав про індикатор роботи диску на ноутбуках і все ж не можу зрозуміти - для чого вони взагалі потрібні і що в них корисного.<br/>
 Не підкажете, чому це так важливо для вас?</p></div>,
 <div class="product-comment__body"><!-- --

In [116]:
soup.findAll('div' ,class_='product-comment__body')[-1].p.text

'Відмінний ноут за цю ціну)'

In [96]:
soup.findAll('div' ,class_='product-comment__body')[-1].findAll('dl', class_='product-comment__essentials')

[<dl class="product-comment__essentials"><dt class="product-comment__essentials-label"> Достоинства: </dt><dd>Ціна)</dd></dl>,
 <dl class="product-comment__essentials"><dt class="product-comment__essentials-label"> Недостатки: </dt><dd>Немає як за цю ціну</dd></dl>]

In [100]:
soup.findAll('div' ,class_='product-comment__body')[-1].findAll('dl', class_='product-comment__essentials')[0].find('dd').text

'Ціна)'

In [95]:
soup.findAll('div' ,class_='product-comment__body')[-1].find('dd')

<dd>Ціна)</dd>

In [15]:
s = "{&q;G.https://common-api.rozetka.com.ua/v2/fat-menu/basic?front-type=xl&a;lang=ru?&q;:{&q;body&q;:{&q;data&q;:{&q;2416&q;:{&q;id&q;:2416,&q;title&q;:&q;Ноутбуки и компьютеры&q;,&q;level&q;:1,&q;order&q;:1,&q;parent_id&q;:0,&q;category_id&q;:80253,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/computers-notebooks/c80253/&q;,&q;outer_link&q;:false},&q;3361&q;:{&q;id&q;:3361,&q;title&q;:&q;Смартфоны, ТВ и электроника&q;,&q;level&q;:1,&q;order&q;:2,&q;parent_id&q;:0,&q;category_id&q;:4627949,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/telefony-tv-i-ehlektronika/c4627949/&q;,&q;outer_link&q;:false},&q;4306&q;:{&q;id&q;:4306,&q;title&q;:&q;Бытовая техника&q;,&q;level&q;:1,&q;order&q;:3,&q;parent_id&q;:0,&q;category_id&q;:80025,&q;on_subdomain&q;:1,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://bt.rozetka.com.ua/&q;,&q;outer_link&q;:false},&q;5300&q;:{&q;id&q;:5300,&q;title&q;:&q;Товары для дома&q;,&q;level&q;:1,&q;order&q;:4,&q;parent_id&q;:0,&q;category_id&q;:2394287,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/tovary-dlya-doma/c2394287/&q;,&q;outer_link&q;:false},&q;6700&q;:{&q;id&q;:6700,&q;title&q;:&q;Инструменты и автотовары&q;,&q;level&q;:1,&q;order&q;:5,&q;parent_id&q;:0,&q;category_id&q;:4627858,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/instrumenty-i-avtotovary/c4627858/&q;,&q;outer_link&q;:false},&q;7806&q;:{&q;id&q;:7806,&q;title&q;:&q;Сантехника и ремонт&q;,&q;level&q;:1,&q;order&q;:6,&q;parent_id&q;:0,&q;category_id&q;:4628418,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/santekhnika-i-remont/c4628418/&q;,&q;outer_link&q;:false},&q;8261&q;:{&q;id&q;:8261,&q;title&q;:&q;Дача, сад и огород&q;,&q;level&q;:1,&q;order&q;:7,&q;parent_id&q;:0,&q;category_id&q;:2394297,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/dacha-sad-ogorod/c2394297/&q;,&q;outer_link&q;:false},&q;9017&q;:{&q;id&q;:9017,&q;title&q;:&q;Спорт и увлечения&q;,&q;level&q;:1,&q;order&q;:8,&q;parent_id&q;:0,&q;category_id&q;:4627893,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/sport-i-uvlecheniya/c4627893/&q;,&q;outer_link&q;:false},&q;10515&q;:{&q;id&q;:10515,&q;title&q;:&q;Одежда, обувь и украшения&q;,&q;level&q;:1,&q;order&q;:9,&q;parent_id&q;:0,&q;category_id&q;:1162030,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/shoes_clothes/c1162030/&q;,&q;outer_link&q;:false},&q;12258&q;:{&q;id&q;:12258,&q;title&q;:&q;Красота и здоровье&q;,&q;level&q;:1,&q;order&q;:10,&q;parent_id&q;:0,&q;category_id&q;:4629305,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/krasota-i-zdorovje/c4629305/&q;,&q;outer_link&q;:false},&q;13224&q;:{&q;id&q;:13224,&q;title&q;:&q;Детские товары&q;,&q;level&q;:1,&q;order&q;:11,&q;parent_id&q;:0,&q;category_id&q;:88468,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/kids/c88468/&q;,&q;outer_link&q;:false},&q;14127&q;:{&q;id&q;:14127,&q;title&q;:&q;Канцтовары и книги&q;,&q;level&q;:1,&q;order&q;:12,&q;parent_id&q;:0,&q;category_id&q;:4625734,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/office-school-books/c4625734/&q;,&q;outer_link&q;:false},&q;14939&q;:{&q;id&q;:14939,&q;title&q;:&q;Алкогольные напитки и продукты&q;,&q;level&q;:1,&q;order&q;:13,&q;parent_id&q;:0,&q;category_id&q;:4626923,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/alkoholnie-napitki-i-produkty/c4626923/&q;,&q;outer_link&q;:false},&q;15422&q;:{&q;id&q;:15422,&q;title&q;:&q;Товары для бизнеса&q;,&q;level&q;:1,&q;order&q;:14,&q;parent_id&q;:0,&q;category_id&q;:4627851,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/tovary-dlya-biznesa/c4627851/&q;,&q;outer_link&q;:false},&q;15954&q;:{&q;id&q;:15954,&q;title&q;:&q;Услуги и сервисы&q;,&q;level&q;:1,&q;order&q;:15,&q;parent_id&q;:0,&q;category_id&q;:4630262,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/uslugi-i-servisi/c4630262/&q;,&q;outer_link&q;:false},&q;17191&q;:{&q;id&q;:17191,&q;title&q;:&q;Космические скидки до 50%&q;,&q;level&q;:1,&q;order&q;:17,&q;parent_id&q;:0,&q;category_id&q;:null,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/promo/cosmicsale/&q;,&q;outer_link&q;:true},&q;18085&q;:{&q;id&q;:18085,&q;title&q;:&q;Продовольственные наборы&q;,&q;level&q;:1,&q;order&q;:16,&q;parent_id&q;:0,&q;category_id&q;:4660608,&q;on_subdomain&q;:0,&q;target_blank&q;:0,&q;manual_url&q;:&q;https://rozetka.com.ua/prodovolstvennie-nabori/c4660608/&q;,&q;outer_link&q;:false}}},&q;headers&q;:{&q;server&q;:[&q;nginx/1.16.1&q;],&q;date&q;:[&q;Wed, 08 Apr 2020 20:33:37 GMT&q;],&q;content-type&q;:[&q;application/json; charset=UTF-8&q;],&q;transfer-encoding&q;:[&q;chunked&q;],&q;connection&q;:[&q;keep-alive&q;],&q;keep-alive&q;:[&q;timeout=20&q;],&q;vary&q;:[&q;Accept-Encoding&q;],&q;domain&q;:[&q;api, common-api&q;]},&q;status&q;:200,&q;statusText&q;:&q;OK&q;,&q;url&q;:&q;https://common-api.rozetka.com.ua/v2/fat-menu/basic?front-type=xl&a;lang=ru&q;},&q;G.https://product-api.rozetka.com.ua/v3/goods/get-main?front-type=xl&a;id=181862304&a;is_canonical=0&a;lang=ru?&q;:{&q;body&q;:{&q;data&q;:{&q;id&q;:181862304,&q;title&q;:&q;Dell Inspiron 3582 (I35C445DIL-73B) Black&q;,&q;price&q;:&q;4999&q;,&q;old_price&q;:&q;5499&q;,&q;price_pcs&q;:&q;178.54&q;,&q;href&q;:&q;https://rozetka.com.ua/dell_i35c445dil_73b/p181862304/&q;,&q;name&q;:&q;dell_i35c445dil_73b&q;,&q;comments_amount&q;:9,&q;sell_status&q;:&q;available&q;,&q;status&q;:&q;active&q;,&q;status_inherited&q;:&q;active&q;,&q;comments_mark&q;:&q;3.5&q;,&q;category_id&q;:80004,&q;seller_id&q;:5,&q;merchant_id&q;:1,&q;group_id&q;:0,&q;producer_id&q;:7,&q;docket&q;:&q;Экран 15.6\&q; TN+film (1366x768) WXGA HD, глянцевый с антибликовым покрытием / Intel Celeron N4000 (1.1 - 2.6 ГГц) / RAM 4 ГБ / HDD 500 ГБ / Intel UHD 600 / DVD+/-RW / Wi-Fi / Bluetooth / веб-камера / Linux / 2.28 кг / черный&q;,&q;state&q;:&q;new&q;,&q;title_short&q;:&q;&q;,&q;promo_title_part&q;:&q;&q;,&q;mpath&q;:&q;.80253.80004.&q;,&q;breadcrumbs&q;:[{&q;id&q;:80253,&q;title&q;:&q;Компьютеры и ноутбуки&q;,&q;href&q;:&q;https://rozetka.com.ua/computers-notebooks/c80253/&q;},{&q;id&q;:80004,&q;title&q;:&q;Ноутбуки&q;,&q;href&q;:&q;https://rozetka.com.ua/notebooks/c80004/&q;},{&q;id&q;:80004,&q;title&q;:&q;Ноутбуки Dell&q;,&q;href&q;:&q;https://rozetka.com.ua/notebooks/c80004/producer=dell/&q;}],&q;brand&q;:&q;Dell&q;,&q;brand_id&q;:7,&q;brand_name&q;:&q;dell&q;,&q;brand_logo&q;:&q;https://i2.rozetka.ua/producers/0/24.jpg&q;,&q;last_category&q;:{&q;id&q;:80004,&q;title&q;:&q;Ноутбуки&q;,&q;href&q;:&q;https://rozetka.com.ua/notebooks/c80004/&q;,&q;rz_mpath&q;:&q;.80253.80004.&q;},&q;tag&q;:{&q;name&q;:&q;popularity&q;,&q;title&q;:&q;Топ продаж&q;,&q;priority&q;:4},&q;is_markdown&q;:0,&q;article&q;:&q;I35C445DIL-73B&q;,&q;loyalty&q;:{&q;pl_bonus_charge_pcs&q;:0,&q;pl_bonus_not_allowed_pcs&q;:true,&q;pl_use_instant_bonus&q;:false,&q;pl_comment_bonus_charge&q;:0,&q;pl_comment_video_bonus_charge&q;:0},&q;has_alternate&q;:true,&q;discount&q;:9,&q;mark_percent&q;:&q;70%&q;,&q;metadata&q;:{&q;title&q;:&q;&q;,&q;description&q;:&q;&q;,&q;keywords&q;:&q;&q;},&q;adult_alert&q;:null,&q;warning&q;:[&q;Внимание! Все ноутбуки имеют русские буквы на клавиатуре и сопровождаются официальной гарантией от производителя! Перед установкой программного обеспечения просьба проверить совместимость с устройством, указанные на упаковке, документации или на веб-сайте&q;,&q;Характеристики и комплектация товара могут изменяться производителем без уведомления.&l;br&g; * Обзор подготовлен на базе одной из моделей серии. Точные спецификации смотрите во вкладке \&q;Характеристики\&q;.&q;],&q;images&q;:[{&q;base_action&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912703.jpg&q;,&q;width&q;:240,&q;height&q;:154},&q;preview&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912640.jpg&q;,&q;width&q;:140,&q;height&q;:90},&q;small&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912843.jpg&q;,&q;width&q;:40,&q;height&q;:25},&q;medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912794.jpg&q;,&q;width&q;:80,&q;height&q;:51},&q;large&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912549.jpg&q;,&q;width&q;:200,&q;height&q;:128},&q;big_tile&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912500.jpg&q;,&q;width&q;:400,&q;height&q;:257},&q;big&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912437.jpg&q;,&q;width&q;:720,&q;height&q;:463},&q;mobile_medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912899.jpg&q;,&q;width&q;:100,&q;height&q;:64},&q;mobile_large&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912871.jpg&q;,&q;width&q;:170,&q;height&q;:109},&q;original&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912374.jpg&q;,&q;width&q;:983,&q;height&q;:633},&q;id&q;:16723912374},{&q;base_action&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913291.jpg&q;,&q;width&q;:240,&q;height&q;:148},&q;preview&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913263.jpg&q;,&q;width&q;:140,&q;height&q;:86},&q;small&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913403.jpg&q;,&q;width&q;:40,&q;height&q;:24},&q;medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913368.jpg&q;,&q;width&q;:80,&q;height&q;:49},&q;large&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913186.jpg&q;,&q;width&q;:200,&q;height&q;:123},&q;big_tile&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913151.jpg&q;,&q;width&q;:400,&q;height&q;:247},&q;big&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913116.jpg&q;,&q;width&q;:720,&q;height&q;:446},&q;mobile_medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913459.jpg&q;,&q;width&q;:100,&q;height&q;:61},&q;mobile_large&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913438.jpg&q;,&q;width&q;:170,&q;height&q;:105},&q;original&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913053.jpg&q;,&q;width&q;:984,&q;height&q;:610},&q;id&q;:16723913053},{&q;base_action&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913872.jpg&q;,&q;width&q;:240,&q;height&q;:169},&q;preview&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913844.jpg&q;,&q;width&q;:140,&q;height&q;:98},&q;small&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913956.jpg&q;,&q;width&q;:40,&q;height&q;:28},&q;medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913942.jpg&q;,&q;width&q;:80,&q;height&q;:56},&q;large&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913767.jpg&q;,&q;width&q;:200,&q;height&q;:141},&q;big_tile&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913739.jpg&q;,&q;width&q;:400,&q;height&q;:282},&q;big&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913704.jpg&q;,&q;width&q;:720,&q;height&q;:508},&q;mobile_medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914026.jpg&q;,&q;width&q;:100,&q;height&q;:70},&q;mobile_large&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913991.jpg&q;,&q;width&q;:170,&q;height&q;:119},&q;original&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723913/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723913627.jpg&q;,&q;width&q;:992,&q;height&q;:700},&q;id&q;:16723913627},{&q;base_action&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914376.jpg&q;,&q;width&q;:240,&q;height&q;:238},&q;preview&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914362.jpg&q;,&q;width&q;:140,&q;height&q;:139},&q;small&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914544.jpg&q;,&q;width&q;:40,&q;height&q;:39},&q;medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914502.jpg&q;,&q;width&q;:80,&q;height&q;:79},&q;large&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914271.jpg&q;,&q;width&q;:200,&q;height&q;:198},&q;big_tile&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914208.jpg&q;,&q;width&q;:352,&q;height&q;:350},&q;big&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914159.jpg&q;,&q;width&q;:634,&q;height&q;:630},&q;mobile_medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914621.jpg&q;,&q;width&q;:100,&q;height&q;:99},&q;mobile_large&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914586.jpg&q;,&q;width&q;:170,&q;height&q;:168},&q;original&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914082.jpg&q;,&q;width&q;:999,&q;height&q;:992},&q;id&q;:16723914082},{&q;base_action&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914964.jpg&q;,&q;width&q;:240,&q;height&q;:168},&q;preview&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914929.jpg&q;,&q;width&q;:140,&q;height&q;:98},&q;small&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915048.jpg&q;,&q;width&q;:40,&q;height&q;:28},&q;medium&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915020.jpg&q;,&q;width&q;:80,&q;height&q;:56},&q;large&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914859.jpg&q;,&q;width&q;:200,&q;height&q;:140},&q;big_tile&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914831.jpg&q;,&q;width&q;:400,&q;height&q;:280},&q;big&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914796.jpg&q;,&q;width&q;:720,&q;height&q;:504},&q;mobile_medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915118.jpg&q;,&q;width&q;:100,&q;height&q;:70},&q;mobile_large&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915090.jpg&q;,&q;width&q;:170,&q;height&q;:119},&q;original&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723914/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723914740.jpg&q;,&q;width&q;:990,&q;height&q;:694},&q;id&q;:16723914740},{&q;base_action&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915377.jpg&q;,&q;width&q;:240,&q;height&q;:168},&q;preview&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915335.jpg&q;,&q;width&q;:140,&q;height&q;:98},&q;small&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915482.jpg&q;,&q;width&q;:40,&q;height&q;:28},&q;medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915454.jpg&q;,&q;width&q;:80,&q;height&q;:56},&q;large&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915272.jpg&q;,&q;width&q;:200,&q;height&q;:140},&q;big_tile&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915251.jpg&q;,&q;width&q;:400,&q;height&q;:280},&q;big&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915209.jpg&q;,&q;width&q;:720,&q;height&q;:504},&q;mobile_medium&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915587.jpg&q;,&q;width&q;:100,&q;height&q;:70},&q;mobile_large&q;:{&q;url&q;:&q;https://i2.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915517.jpg&q;,&q;width&q;:170,&q;height&q;:119},&q;original&q;:{&q;url&q;:&q;https://i1.rozetka.ua/goods/16723915/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723915167.jpg&q;,&q;width&q;:1024,&q;height&q;:717},&q;id&q;:16723915167}],&q;videos&q;:[],&q;video_presentation&q;:[],&q;description&q;:{&q;text&q;:&q;&l;p&g;Универсальный 15.6-дюймовый ноутбук по доступной цене &l;b&g;Dell Inspiron 3582&l;/b&g;, оснащенный развлекательными возможностями, которые обеспечивают великолепное качество воспроизведения мультимедийных материалов.&l;/p&g;&l;p&g;&l;b&g;Переносите фото и видео:&l;/b&g; устройство считывания карт памяти SD упрощает перенос файлов с камеры на ноутбук. &l;/p&g;&l;p&g;&l;b&g;Связь в формате высокой четкости:&l;/b&g; общайтесь с друзьями на расстоянии, записывайте видеоролики для YouTube, участвуйте в веб-конференциях с помощью веб-камеры высокой четкости. &l;/p&g;&l;p&g;&l;b&g;Устойчивость к высоким температурам.&l;/b&g; Dell тестируют ноутбуки Inspiron на устойчивость к непродолжительному пребыванию в условиях высокой температуры (например, если вы оставите его в шкафчике спортзала или закрытой машине).&l;/p&g;&l;p&g;&l;b&g;Надежные шарниры.&l;/b&g; Dell тестируют шарниры ноутбука Inspiron, чтобы обеспечить их идеальное состояние даже после открытия и закрытия крышки 20 000 раз.&l;/p&g;&l;p&g;&l;b&g;Безотказные клавиши.&l;/b&g; Тестируя клавиатуру, Dell проверяют ее способность безошибочно выдерживать 10 миллионов нажатий часто используемых клавиш и не менее 1 миллиона нажатий кнопок сенсорной панели. Ничто не помешает вам закончить свой роман.&l;/p&g;&l;p&g;&l;b&g;Надежные кнопки.&l;/b&g; Неважно, как часто вы будете использовать кнопки питания и мультимедийные кнопки, Dell протестировали их на гарантированную устойчивость к 40 000 нажатий.&l;/p&g;&l;p&g;&l;b&g;Абсолютная прочность.&l;/b&g; Это может показаться чрезмерной перестраховкой, но Dell гарантируют, что даже после 25 000 циклов проверки основания и крышки ноутбука Inspiron на изгиб, его внутренние компоненты надежно защищены.&l;/p&g;&q;,&q;html&q;:null},&q;seo&q;:{&q;BreadcrumbList&q;:{&q;@context&q;:&q;http://schema.org&q;,&q;@type&q;:&q;BreadcrumbList&q;,&q;ItemListElement&q;:[{&q;@type&q;:&q;ListItem&q;,&q;position&q;:1,&q;item&q;:{&q;@id&q;:&q;https://rozetka.com.ua/computers-notebooks/c80253/&q;,&q;name&q;:&q;Компьютеры и ноутбуки&q;}},{&q;@type&q;:&q;ListItem&q;,&q;position&q;:2,&q;item&q;:{&q;@id&q;:&q;https://rozetka.com.ua/notebooks/c80004/&q;,&q;name&q;:&q;Ноутбуки&q;}},{&q;@type&q;:&q;ListItem&q;,&q;position&q;:3,&q;item&q;:{&q;@id&q;:&q;https://rozetka.com.ua/notebooks/c80004/producer=dell/&q;,&q;name&q;:&q;Ноутбуки Dell&q;}}]},&q;Product&q;:{&q;@context&q;:&q;http://schema.org&q;,&q;@type&q;:&q;Product&q;,&q;sku&q;:&q;181862304&q;,&q;url&q;:&q;https://rozetka.com.ua/dell_i35c445dil_73b/p181862304/&q;,&q;name&q;:&q;Dell Inspiron 3582 (I35C445DIL-73B) Black&q;,&q;image&q;:&q;https://i2.rozetka.ua/goods/16723912/copy_dell_i3582c4h5dil_bk_5e39b3d6ee103_images_16723912703.jpg&q;,&q;description&q;:&q;Универсальный 15.6-дюймовый ноутбук по доступной цене Dell Inspiron 3582, оснащенный развлекательными возможностями, которые обеспечивают великолепное качество воспроизведения мультимедийных материалов.Переносите фото и видео: устройство считывания карт памяти SD упрощает перенос файлов с камеры на ноутбук. Связь в формате высокой четкости: общайтесь с друзьями на расстоянии, записывайте видеоролики для YouTube, участвуйте в веб-конференциях с помощью веб-камеры высокой четкости. Устойчивость к высоким температурам. Dell тестируют ноутбуки Inspiron на устойчивость к непродолжительному пребыванию в условиях высокой температуры (например, если вы оставите его в шкафчике спортзала или закрытой машине).Надежные шарниры. Dell тестируют шарниры ноутбука Inspiron, чтобы обеспечить их идеальное состояние даже после открытия и закрытия крышки 20 000 раз.Безотказные клавиши. Тестируя клавиатуру, Dell проверяют ее способность безошибочно выдерживать 10 миллионов нажатий часто используемых клавиш и не менее 1 миллиона нажатий кнопок сенсорной панели. Ничто не помешает вам закончить свой роман.Надежные кнопки. Неважно, как часто вы будете использовать кнопки питания и мультимедийные кнопки, Dell протестировали их на гарантированную устойчивость к 40 000 нажатий.Абсолютная прочность. Это может показаться чрезмерной перестраховкой, но Dell гарантируют, что даже после 25 000 циклов проверки основания и крышки ноутбука Inspiron на изгиб, его внутренние компоненты надежно защищены.&q;,&q;itemCondition&q;:&q;https://schema.org/NewCondition&q;,&q;brand&q;:{&q;@type&q;:&q;Brand&q;,&q;name&q;:&q;Dell&q;,&q;url&q;:&q;https://rozetka.com.ua/producer/dell/&q;},&q;offers&q;:{&q;@type&q;:&q;Offer&q;,&q;availability&q;:&q;http://schema.org/InStock&q;,&q;url&q;:&q;https://rozetka.com.ua/dell_i35c445dil_73b/p181862304/&q;,&q;price&q;:&q;4999.00&q;,&q;priceCurrency&q;:&q;UAH&q;,&q;priceValidUntil&q;:&q;2021-04-08T23:21:14&q;}},&q;Videos&q;:null},&q;short_title&q;:&q;&q;,&q;size_chart_link&q;:null,&q;pictograms&q;:{&q;promo&q;:[{&q;id&q;:26839,&q;title&q;:&q;Космические скидки до 50%&q;,&q;location&q;:&q;in_central_block&q;,&q;description&q;:&q;&l;p&g;Космические скидки до 50%&l;/p&g;&l;p&g;&l;a href=\&q;https://rozetka.com.ua/promo/cosmicsale/\&q; target=\&q;_blank\&q;&g;Подробнее&l;/a&g;.&l;/p&g;&q;,&q;is_specials_tag&q;:true,&q;link&q;:null,&q;link_title&q;:null,&q;src&q;:&q;https://i1.rozetka.ua/goods/tags/85/85999.png&q;},{&q;id&q;:26821,&q;title&q;:&q;Возвращаем 5% на бонусный счет при оплате картой Visa&q;,&q;location&q;:&q;in_central_block&q;,&q;description&q;:&q;&l;p&g;Возвращаем 5% на бонусный счет при оплате картой Visa&l;/p&g;&l;p&g;&l;a href=\&q;https://rozetka.com.ua/news-articles-promotions/promotions/visa.html\&q; target=\&q;_blank\&q;&g;Подробнее&l;/a&g;.&l;/p&g;&q;,&q;is_specials_tag&q;:true,&q;link&q;:null,&q;link_title&q;:null,&q;src&q;:&q;https://i1.rozetka.ua/goods/tags/85/85963.png&q;},{&q;id&q;:10167,&q;title&q;:&q;Дарим 100 грн за распаковку&q;,&q;location&q;:&q;in_central_block&q;,&q;description&q;:&q;&l;p&g;Дарим 100 грн за распаковку товара.&l;/p&g;&l;p&g;Автор каждого видеоролика, который редакция сайта сочтет качественным, получит поощрительный промокод на 100 грн&l;/p&g;&l;p&g;&l;a href=\&q;//rozetka.com.ua/100grn/\&q; target=\&q;_blank\&q;&g;Подробнее&l;/a&g;&l;/p&g;&q;,&q;is_specials_tag&q;:true,&q;link&q;:null,&q;link_title&q;:null,&q;src&q;:&q;https://i1.rozetka.ua/goods/tags/71/71424.png&q;},{&q;id&q;:3876,&q;title&q;:&q;Кредит 0,01% на 3 месяца!&q;,&q;location&q;:&q;in_central_block&q;,&q;description&q;:&q;&l;p&g;Для оформления кредита достаточно лишь предоставить свои паспортные данные и идентификационный код. &l;br/&g;Возможно досрочное погашение без комиссий. &l;br/&g;Оформление кредита возможно по всей Украине!&l;/p&g;&l;p&g;Подарки не предоставляются при покупке товара в 0,01% кредит или оплату частями.&l;/p&g;&q;,&q;is_specials_tag&q;:true,&q;link&q;:null,&q;link_title&q;:null,&q;src&q;:&q;https://i1.rozetka.ua/goods/tags/43/43104.jpg&q;}],&q;technical&q;:[{&q;id&q;:935,&q;title&q;:&q;Intel Celeron&q;,&q;location&q;:&q;in_central_block&q;,&q;description&q;:&q;&l;p&g;Легкая работа в многозадачной среде: процессор Intel® Celeron® — отличный вариант для начала работы с ПК. Работаете ли вы с офисными проектами, играете в игры или общаетесь в чате с друзьями и семьей, вы можете делать это все мгновенно.&l;/p&g;&q;,&q;is_specials_tag&q;:false,&q;link&q;:&q;http://rozetka.com.ua/intel_processor/#power1&q;,&q;link_title&q;:&q;Подробнее&q;,&q;src&q;:&q;https://i1.rozetka.ua/goods/tags/74/74653.jpg&q;},{&q;id&q;:1074,&q;title&q;:&q;RAM 4GB&q;,&q;location&q;:&q;in_central_block&q;,&q;description&q;:null,&q;is_specials_tag&q;:false,&q;link&q;:null,&q;link_title&q;:null,&q;src&q;:&q;https://i1.rozetka.ua/goods/tags/83/83700.png&q;},{&q;id&q;:1124,&q;title&q;:&q;HDD 500GB&q;,&q;location&q;:&q;in_central_block&q;,&q;description&q;:null,&q;is_specials_tag&q;:false,&q;link&q;:null,&q;link_title&q;:null,&q;src&q;:&q;https://i1.rozetka.ua/goods/tags/84/84029.png&q;},{&q;id&q;:970,&q;title&q;:&q;2 cores&q;,&q;location&q;:&q;in_central_block&q;,&q;description&q;:null,&q;is_specials_tag&q;:false,&q;link&q;:null,&q;link_title&q;:null,&q;src&q;:&q;https://i1.rozetka.ua/goods/tags/74/74001.png&q;},{&q;id&q;:4905,&q;title&q;:&q;DDR4&q;,&q;location&q;:&q;in_central_block&q;,&q;description&q;:null,&q;is_specials_tag&q;:false,&q;link&q;:null,&q;link_title&q;:null,&q;src&q;:&q;https://i1.rozetka.ua/goods/tags/79/79953.png&q;}],&q;label&q;:{&q;id&q;:26803,&q;title&q;:&q;Бесплатная доставка покупок с Rozetka&q;,&q;location&q;:&q;dominate_price_tag&q;,&q;description&q;:&q;&l;p&g;Бесплатная доставка покупок с Rozetka&l;/p&g;&l;p&g;&l;a href=\&q;https://rozetka.com.ua/news-articles-promotions/promotions/delivery.html\&q; target=\&q;_blank\&q;&g;Выберите удобный способ доставки&l;/a&g;&l;/p&g;&q;,&q;is_specials_tag&q;:true,&q;link&q;:null,&q;link_title&q;:null,&q;src&q;:&q;https://i2.rozetka.ua/goods/tags/85/85933.png&q;}},&q;isCanonical&q;:false}},&q;headers&q;:{&q;server&q;:[&q;nginx/1.16.1&q;],&q;date&q;:[&q;Wed, 08 Apr 2020 20:33:37 GMT&q;],&q;content-type&q;:[&q;application/json; charset=UTF-8&q;],&q;transfer-encoding&q;:[&q;chunked&q;],&q;connection&q;:[&q;keep-alive&q;],&q;keep-alive&q;:[&q;timeout=20&q;],&q;vary&q;:[&q;Accept-Encoding&q;],&q;domain&q;:[&q;product-api, product-api&q;]},&q;status&q;:200,&q;statusText&q;:&q;OK&q;,&q;url&q;:&q;https://product-api.rozetka.com.ua/v3/goods/get-main?front-type=xl&a;id=181862304&a;is_canonical=0&a;lang=ru&q;},&q;G.https://xl-static.rozetka.com.ua/assets/i18n/ru.json?&q;:{&q;body&q;:{&q;lang&q;:&q;ru&q;,&q;header-location-caption&q;:&q;Выбор города поможет предоставить актуальную информацию о наличии товара, его цены и способов доставки в вашем городе! Это поможет сохранить больше свободного времени для вас!&q;,&q;product_photo_gallery_title&q;:&q;Фотографии {{productTitle}}&q;,&q;cart.one&q;:&q;В корзине {{count}} товар&q;,&q;cart.few&q;:&q;В корзине {{count}} товара&q;,&q;cart.other&q;:&q;В корзине {{count}} товаров&q;,&q;checkout.one&q;:&q;{{count}} товар&q;,&q;checkout.few&q;:&q;{{count}} товара&q;,&q;checkout.other&q;:&q;{{count}} товаров&q;,&q;comparison.one&q;:&q;У вас {{count}} товар в сравненнии&q;,&q;comparison.few&q;:&q;У вас {{count}} товара в сравненнии&q;,&q;comparison.other&q;:&q;У вас {{count}} товаров в сравненнии&q;,&q;wishlist.one&q;:&q;У вас {{count}} товар в списке желаний&q;,&q;wishlist.few&q;:&q;У вас {{count}} товара в списке желаний&q;,&q;wishlist.other&q;:&q;У вас {{count}} товаров в списке желаний&q;,&q;from-to&q;:&q; с {{ from }} до {{ to }}&q;,&q;show.more.one&q;:&q;еще {{count}} товар&q;,&q;show.more.few&q;:&q;еще {{count}} товара&q;,&q;show.more.other&q;:&q;еще {{count}} товаров&q;,&q;comments.one&q;:&q;{{count}} отзыв&q;,&q;comments.few&q;:&q;{{count}} отзыва&q;,&q;comments.other&q;:&q;{{count}} отзывов&q;,&q;selected.one&q;:&q;Выбран {{count}} товар&q;,&q;selected.few&q;:&q;Выбрано {{count}} товара&q;,&q;selected.other&q;:&q;Выбрано {{count}} товаров&q;,&q;selected_mobile.one&q;:&q;Найден {{count}} товар&q;,&q;selected_mobile.few&q;:&q;Найдено {{count}} товара&q;,&q;selected_mobile.other&q;:&q;Найдено {{count}} товаров&q;,&q;bonuses.one&q;:&q;{{count}} бонусная&q;,&q;bonuses.few&q;:&q;{{count}} бонусных&q;,&q;bonuses.other&q;:&q;{{count}} бонусных&q;,&q;replies.one&q;:&q;{{count}} ответ&q;,&q;replies.few&q;:&q;{{count}} ответа&q;,&q;replies.other&q;:&q;{{count}} ответов&q;,&q;services.one&q;:&q;{{count}} услугу&q;,&q;services.few&q;:&q;{{count}} услуги&q;,&q;services.other&q;:&q;{{count}} услуг&q;,&q;Пользователь не зарегистрирован&q;:&q;Пользователь с логином {{login}} не зарегистрирован&q;,&q;producer&q;:&q;Бренд&q;,&q;price&q;:&q;Цена&q;,&q;Более брендов&q;:&q;Более {{ count }} брендов&q;,&q;Более украшений&q;:&q;Более {{ count }} типов украшений&q;,&q;unavailable&q;:&q;Нет в наличии&q;,&q;waiting_for_supply&q;:&q;Ожидается&q;,&q;limited&q;:&q;Заканчивается&q;,&q;available&q;:&q;Есть в наличии&q;,&q;archive&q;:&q;Архивный&q;,&q;out_of_stock&q;:&q;Закончился&q;,&q;hidden&q;:&q;Скрытый&q;,&q;checkout error empty orders&q;:&q;Упс, заказ не сформирован, мы сохранили его в корзине. Перезагрузите страницу или попробуйте оформить заказ позже.&q;,&q;checkout error save order&q;:&q;Упс, заказ не оформлен, мы сохранили его в корзине. Перезагрузите страницу или попробуйте оформить заказ позже.&q;,&q;order passport need&q;:&q;Получение заказа от 15 000 грн только по паспорту (Приказ НБУ 99-3)&q;,&q;checkout need passport recipient&q;:&q;Внимание! Получение заказа по паспорту. Введите фамилию, имя, отчество и мобильный номер телефона получателя заказа&q;,&q;checkout.order_status.error&q;:&q;Оплата не успешна&q;,&q;checkout.order_status.ok&q;:&q;Оплата успешна&q;,&q;checkout.order_status.wait&q;:&q;Ожидаем ответ от банка&q;,&q;checkout.payment.nonCash.message&q;:&q;Внимание! Для оплаты выбранным способом требуется ввести код ЕГРПОУ юридического лица, с расчетного счета которого будет произведена оплата&q;,&q;checkout_thank_you_page.one&q;:&q;{{count}} заказ&q;,&q;checkout_thank_you_page.few&q;:&q;{{count}} заказа&q;,&q;checkout_thank_you_page.other&q;:&q;{{count}} заказов&q;,&q;goods_for_the_amount.one&q;:&q;{{count}} товар на сумму&q;,&q;goods_for_the_amount.few&q;:&q;{{count}} товара на сумму&q;,&q;goods_for_the_amount.other&q;:&q;{{count}} товаров на сумму&q;,&q;new&q;:&q;Новый&q;,&q;used&q;:&q;Б/У&q;,&q;register.password.notification&q;:&q;Пароль должен быть не менее 6 символов, содержать цифры и заглавные буквы и не должен совпадать с именем и эл.почтой&q;,&q;order_status.one_click&q;:&q;Новый заказ в 1 клик&q;,&q;order_status.new&q;:&q;Новый заказ&q;,&q;order_status.cancel-user&q;:&q;Отменен&q;,&q;order_status.wait-client-approve&q;:&q;Ожидает подтверждения клиентом&q;,&q;order_status.processed-automatically&q;:&q;Обработан автоматически&q;,&q;order_status.progress&q;:&q;Обрабатывается менеджером&q;,&q;order_status.wait-payment&q;:&q;Ожидает получения оплаты&q;,&q;order_status.cancel-contact&q;:&q;Не удалось связаться с клиентом&q;,&q;order_status.cancel-noavailable&q;:&q;Товара нет в наличии&q;,&q;order_status.cancel-nopay&q;:&q;Отсутстует оплата&q;,&q;order_status.cancel-took&q;:&q;Клиент не забрал товар&q;,&q;order_status.cancel-test&q;:&q;Тестовый заказ&q;,&q;order_status.expired_reserve&q;:&q;Истек срок резерва&q;,&q;order_status.completing&q;:&q;Комплектуется&q;,&q;order_status.completing-carrier&q;:&q;Комплектуется перевозчиком&q;,&q;order_status.to-be-completed&q;:&q;Требует доукомплектации&q;,&q;order_status.delivery-planned&q;:&q;Запланирована доставка&q;,&q;order_status.to-delivry&q;:&q;Передан в службу доставки&q;,&q;order_status.to-poland-delivery&q;:&q;Передан в службу доставки в Польше&q;,&q;order_status.to-customs-control&q;:&q;На таможенном контроле&q;,&q;order_status.to-ukraine-delivery&q;:&q;Передан в службу доставки в Украине&q;,&q;order_status.planned-to-delivery&q;:&q;Запланирована передача перевозчику&q;,&q;order_status.wait-client&q;:&q;Ожидает клиента в пункте самовывоза&q;,&q;order_status.cancel-nosatisfied-goods&q;:&q;Не устраивает товар&q;,&q;order_status.cancel-nosatisfied-payment&q;:&q; Не устраивают условия оплаты&q;,&q;order_status.cancel-nosatisfied-delivery&q;:&q; Не устраивают условия доставки&q;,&q;order_status.delivering&q;:&q;Доставляется&q;,&q;order_status.give-date-changed&q;:&q;Изменена дата выдачи&q;,&q;order_status.delivery-service-processing&q;:&q;Обрабатывается службой доставки&q;,&q;order_status.returned&q;:&q;Заказ возвращен&q;,&q;order_status.complete&q;:&q;Выполнен&q;,&q;order_status.user_canceled&q;:&q;Отменен пользователем&q;,&q;order_status.canceled-order-copied&q;:&q;Создана копия&q;,&q;order_status.delivery-to-clients-city&q;:&q;Следует в город получателя&q;,&q;order_status.in-clients-city&q;:&q;Находится в городе получателя&q;,&q;order_status.in-post-office&q;:&q;Находится в городе получателя&q;,&q;order_status.issued-to-courier&q;:&q;Выдано курьеру в городе получателя&q;,&q;order_status.received-partially&q;:&q;Получено частично&q;,&q;order_status.changed-of-delivery&q;:&q;Изменены дата или адрес доставки&q;,&q;order_status.planned-to-call&q;:&q;Планируется повторный звонок&q;,&q;order_status.returned-goods&q;:&q;Товар вернулся&q;,&q;order_status.cancel-refund&q;:&q;Произведен возврат средств&q;,&q;order_status.check-in-shop&q;:&q;В очереди на получение&q;,&q;order_status.reserve-expires&q;:&q;Сегодня истекает срок резерва&q;,&q;order_status.refusal&q;:&q;Отказано банком&q;,&q;order_status.issuance-time-modified&q;:&q;Изменено время выдачи&q;,&q;order_status.wait-callcenter-approve&q;:&q;Ожидает модерации КЦ&q;,&q;payments.one&q;:&q;{{count}} платеж на&q;,&q;payments.few&q;:&q;{{count}} платежа на&q;,&q;payments.other&q;:&q;{{count}} платежей на&q;,&q;months.one&q;:&q;{{count}} месяц&q;,&q;months.few&q;:&q;{{count}} месяца&q;,&q;months.other&q;:&q;{{count}} месяцев&q;,&q;product_tab__title&q;:&q;ROZETKA | {{product_name}}. Цена, купить {{product_name}} в Киеве, Харькове, Днепропетровске, Одессе, Запорожье, Львове. {{product_name}}: обзор, описание, продажа&q;,&q;product_tab__description&q;:&q;{{product_name}} купить  на ROZETKA. Оперативная доставка ✈ Гарантия качества ☑ Лучшая цена $&q;,&q;product_tab__keywords&q;:&q;{{product_name}}&q;,&q;product_tab_characteristics_title&q;:&q;ROZETKA | Характеристики {{product_name}}: отзывы, видеообзор, фото. Купить {{product_name}} в Киеве, Харькове, Днепропетровске, Одессе, Запорожье, Львове&q;,&q;product_tab_characteristics_description&q;:&q;Характеристики {{product_name}} купить на ROZETKA. Оперативная доставка ✈ Гарантия качества ☑ Лучшая цена $&q;,&q;product_tab_characteristics_keywords&q;:&q;Характеристики {{product_name}}&q;,&q;product_tab_comments_title&q;:&q;ROZETKA  | Отзывы о {{product_name}}: обсуждение, фото, видео. Купить {{product_name}} в Киеве&q;,&q;product_tab_comments_title_page&q;:&q;Страница {номер}. Отзывы покупателей о {{product_name}} | ROZETKA&q;,&q;product_tab_comments_description&q;:&q;Отзывы о {{product_name}} купить на ROZETKA. Оперативная доставка ✈ Гарантия качества ☑ Лучшая цена $&q;,&q;product_tab_comments_keywords&q;:&q;Отзывы о {{product_name}}&q;,&q;product_tab_delivery_title&q;:&q;ROZETKA | Купить {{product_name}} в кредит, быстрая доставка, гарантия&q;,&q;product_tab_delivery_description&q;:&q;Покупка {{product_name}} – купить на ROZETKA. Оперативная доставка ✈ Гарантия качества ☑ Лучшая цена $&q;,&q;product_tab_delivery_keywords&q;:&q;Доставка {{product_name}}&q;,&q;product_tab_photo_title&q;:&q;ROZETKA | Фото {{product_name}}, купить {{product_name}} в Киеве&q;,&q;product_tab_photo_description&q;:&q;Фото {{product_name}} купить на ROZETKA. Оперативная доставка ✈ Гарантия качества ☑ Лучшая цена $&q;,&q;product_tab_photo_keywords&q;:&q;Фото {{product_name}}&q;,&q;product_tab_review_title&q;:&q;ROZETKA | Видеообзор {{product_name}}, купить {{product_name}} в Киеве&q;,&q;product_tab_review_description&q;:&q;Обзор и видео {{product_name}} купить на ROZETKA. Оперативная доставка ✈ Гарантия качества ☑ Лучшая цена $&q;,&q;product_tab_review_keywords&q;:&q;Обзор {{product_name}}, видео {{product_name}}&q;,&q;product_tab_software_title&q;:&q;ROZETKA | Программное обеспечение для {{product_name}}. Продажа программного обеспечение для {{product_name}} в Киеве&q;,&q;product_tab_software_description&q;:&q;Программное обеспечение для {{product_name}} купить на ROZETKA. Оперативная доставка ✈ Гарантия качества ☑ Лучшая цена $&q;,&q;product_tab_software_keywords&q;:&q;Программное обеспечение для {{product_name}}&q;,&q;product_tab_accessories_title&q;:&q;ROZETKA | Аксессуары для {{product_name}}. Купить аксессуары для {{product_name}} в Киеве, Харькове, Днепропетровске, Одессе, Запорожье, Львове&q;,&q;product_tab_accessories_description&q;:&q;Аксессуары для {{product_name}} на ROZETKA. Оперативная доставка ✈ Гарантия качества ☑ Лучшая цена $&q;,&q;product_tab_accessories_keywords&q;:&q;Аксессуары для {{product_name}}&q;,&q;credit_modal_footer_text&q;:&q;Для оформления кредита необходимо нажать кнопку &a;laquo;Купить&a;raquo;, перейти к&a;nbsp;оформлению заказа, либо продолжить покупки и добавить дополнительный товар в&a;nbsp;&a;laquo;Корзину&a;raquo;. В&a;nbsp;&a;laquo;Корзине&a;raquo; выбрать способ оплаты покупок в&a;nbsp;&a;laquo;Кредит&a;raquo;, заполнить анкету и&a;nbsp;отправить на&a;nbsp;рассмотрение. После этого с&a;nbsp;Вами свяжется сотрудник для уточнения данных.&q;,&q;main_page_metadata_title&q;:&q;Интернет-магазин ROZETKA™: официальный сайт самого популярного онлайн-гипермаркета в Украине&q;,&q;main_page_metadata_description&q;:&q;Интернет-магазин ROZETKA: электроника, одежда и обувь, бытовая техника, автотовары, товары для дома и бизнеса. Покупайте в Розетке: ✓ Официальная гарантия ✓ Доставка по всей Украине ������ ✓ Отзывы покупателей, обзоры и характеристики товаров $ Выгодные цены и скидки %&q;,&q;main_page_metadata_keywords&q;:&q;rozetka, розетка, онлайн-гипермаркет&q;,&q;in_kyiv&q;:&q;Киеве&q;,&q;in_odessa&q;:&q;Одессе&q;,&q;in_brovary&q;:&q;Броварах&q;,&q;kyiv&q;:&q;Киев&q;,&q;odessa&q;:&q;Одесса&q;,&q;brovary&q;:&q;Бровары&q;,&q;visa_text_0&q;:&q;Платежные системы в партнерстве с банками-эмитентами внедряют современные схемы проверки личности держателя карты, чтобы сделать покупки в Интернете более безопасными. Устанавливается специальный пароль для каждой совершаемой операции, и это вселяет в Вас уверенность, что только Вы можете делать такие покупки онлайн. Современные технологические решения необходимы для того, чтобы держатель карты был уверен в безопасности транзакции и в том, что он имеет дело с настоящим (а не «подставным») магазином.&q;,&q;visa_text_1&q;:&q;Одно из таких технологических решений называется Verified by Visa (&a;laquo;Проверено Visa&a;raquo;).&q;,&q;visa_text_2&q;:&q;Verified by Visa &a;mdash; это новая система защиты, которая извещает онлайн-торговцев, участвующих в программе, и банки о том, что Вы являетесь настоящим владельцем карточки, когда Вы совершаете онлайн-покупки. Она позволяет Вам использовать персональный пароль для подтверждения Вашей личности и защиты Вашей карточки Visa, когда Вы используете ее в Интернете, вселяя большую уверенность в совершаемых Вами действиях.&q;,&q;visa_text_3&q;:&q;К технологии Verified by Visa легко подключиться, ее легко использовать, услуга действует для всех карточек Visa.&q;,&q;visa_text_4&q;:&q;Для активации услуги Verified by Visa для Вашей карты необходимо обратиться в Ваш банк.&q;,&q;mastercard_text_0&q;:&q;MasterCard&a;reg; SecureCode&a;trade; &a;mdash; это технология, которая обеспечивает Вашу карту MasterCard&a;reg; или Maestro&a;reg; дополнительным секретным кодом, защищающим ее от несанкционированного использования во время совершения покупок в интернет-магазинах, участвующих в программе.&q;,&q;mastercard_text_1&q;:&q;Чтобы подключить технологию SecureCode, необходимо обратиться в банк, выпустивший Вашу карту. Ваш секретный код будет известен только Вам и Вашему банку. Далее при каждой оплате покупок или услуг в интернет-магазинах, участвующих в программе, Ваш банк предложит Вам ввести данный секретный код, который Вы получите на Ваш мобильный телефон, зарегистрированный в банке. Только после проверки и подтверждения банком данного кода операция по оплате будет произведена. Процедура использования технологии SecureCode так же проста, как введение PIN-кода в банкомате.&q;,&q;mastercard_text_2&q;:&q;Введя правильный SecureCode в процессе покупки, Вы подтверждаете, что являетесь авторизованным владельцем карты. Если введен неправильный SecureCode, покупка не будет совершена. Даже если кому-то известен номер Вашей платежной карты, без SecureCode не удастся оплатить покупку в интернет-магазинах, участвующих в программе.&q;,&q;register.password.hint&q;:&q;Пароль должен быть не менее 6 символов, содержать цифры и латинские буквы, в том числе заглавные, и не должен совпадать с именем и эл. почтой&q;,&q;user_videos&q;:&q;Видео распаковки от покупателя&q;,&q;full_review&q;:&q;Полноценный видеообзор&q;,&q;promo_video&q;:&q;Рекламный ролик&q;,&q;demo_goods&q;:&q;Демонстрация товара (распаковка)&q;,&q;days.one&q;:&q;{{count}} день&q;,&q;days.few&q;:&q;{{count}} дня&q;,&q;days.other&q;:&q;{{count}} дней&q;,&q;found.one&q;:&q;Найден {{count}} товар&q;,&q;found.few&q;:&q;Найдено {{count}} товара&q;,&q;found.other&q;:&q;Найдено {{count}} товаров&q;,&q;add_photo_text&q;:&q;Перетащите файлы сюда или нажмите на кнопку. Добавляйте до 10 изображений в форматах .jpg, .gif, .png, размером файла до 5 МБ&q;,&q;found.more&q;:&q;Найдено более {{count}} товаров&q;,&q;stock_balance_text&q;:&q;осталось&q;,&q;more_items_category&q;:&q;Больше товаров из категории&q;,&q;prolong_order.error&q;:&q;К сожалению, невозможно продлить резерв заказа&q;,&q;prolong_order.ok&q;:&q;Вы продлили резерв заказа до&q;,&q;prolong_order.serverError&q;:&q;Не удалось продлить резерв заказа. Пожалуйста, повторите через&q;,&q;payment_error&q;:&q;К сожалению, оплата {{payment_method_title}} не удалась.&q;,&q;popular_categories&q;:&q;Популярные категории&q;,&q;promo_code_title&q;:&q;Промокод&q;,&q;promo_code_title_discount&q;:&q;на скидку&q;,&q;states&q;:{&q;all&q;:&q;Другие состояния&q;,&q;new&q;:&q;Новый&q;,&q;used&q;:&q;Б/У&q;,&q;refurbished&q;:&q;Восстановленный&q;,&q;markdown&q;:&q;Уценка&q;}},&q;headers&q;:{&q;server&q;:[&q;nginx&q;],&q;date&q;:[&q;Wed, 08 Apr 2020 20:33:37 GMT&q;],&q;content-type&q;:[&q;application/json&q;],&q;content-length&q;:[&q;21865&q;],&q;connection&q;:[&q;keep-alive&q;],&q;keep-alive&q;:[&q;timeout=20&q;],&q;vary&q;:[&q;Accept-Encoding, Origin&q;],&q;access-control-allow-origin&q;:[&q;*&q;],&q;last-modified&q;:[&q;Wednesday, 08-Apr-2020 20:33:38 GMT&q;],&q;cache-control&q;:[&q;no-store, no-cache, must-revalidate, proxy-revalidate, max-age=0&q;],&q;accept-ranges&q;:[&q;bytes&q;],&q;domain&q;:[&q;static, static&q;]},&q;status&q;:200,&q;statusText&q;:&q;OK&q;,&q;url&q;:&q;https://xl-static.rozetka.com.ua/assets/i18n/ru.json&q;},&q;G.https://product-api.rozetka.com.ua/v3/goods/get-characteristic?front-type=xl&a;goodsId=181862304&a;lang=ru?&q;:{&q;body&q;:{&q;data&q;:[{&q;group_id&q;:0,&q;group_order&q;:99999999,&q;groupTitle&q;:&q;&q;,&q;options&q;:[{&q;id&q;:20861,&q;title&q;:&q;Диагональ экрана&q;,&q;order&q;:3,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;15.6\&q; (1366x768) WXGA HD&q;,&q;href&q;:&q;/notebooks/c80004/20861=6308/&q;}]},{&q;id&q;:20862,&q;title&q;:&q;Процессор&q;,&q;order&q;:6,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;Двухъядерный Intel Celeron N4000 (1.1 - 2.6 ГГц)&q;,&q;href&q;:&q;/notebooks/c80004/processor=6279/&q;}]},{&q;id&q;:177846,&q;title&q;:&q;Частота обновления экрана&q;,&q;order&q;:10,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;60 Гц&q;,&q;href&q;:&q;/notebooks/c80004/chastota-obnovleniya-ekrana=60-gts/&q;}]},{&q;id&q;:20863,&q;title&q;:&q;Объем оперативной памяти&q;,&q;order&q;:12,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;4 ГБ&q;,&q;href&q;:&q;/notebooks/c80004/20863=25206/&q;}]},{&q;id&q;:20860,&q;title&q;:&q;Краткие характеристики&q;,&q;order&q;:20,&q;in_short&q;:1,&q;values&q;:[{&q;title&q;:&q;Экран 15.6\&q; TN+film (1366x768) WXGA HD, глянцевый с антибликовым покрытием / Intel Celeron N4000 (1.1 - 2.6 ГГц) / RAM 4 ГБ / HDD 500 ГБ / Intel UHD 600 / DVD+/-RW / Wi-Fi / Bluetooth / веб-камера / Linux / 2.28 кг / черный&q;,&q;href&q;:null}]},{&q;id&q;:20886,&q;title&q;:&q;Операционная система&q;,&q;order&q;:21,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;Linux&q;,&q;href&q;:&q;/notebooks/c80004/20886=6324/&q;}]},{&q;id&q;:21737,&q;title&q;:&q;Цвет&q;,&q;order&q;:42,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;Черный&q;,&q;href&q;:&q;/notebooks/c80004/21737=20378/&q;}]},{&q;id&q;:72560,&q;title&q;:&q;Клавиатура&q;,&q;order&q;:60,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;Без подсветки&q;,&q;href&q;:&q;/notebooks/c80004/72560=305294/&q;}]},{&q;id&q;:20882,&q;title&q;:&q;Объём накопителя&q;,&q;order&q;:66,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;500 ГБ&q;,&q;href&q;:&q;/notebooks/c80004/20882=6317/&q;}]},{&q;id&q;:20868,&q;title&q;:&q;Оптический привод&q;,&q;order&q;:72,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;DVD+/-RW&q;,&q;href&q;:&q;/notebooks/c80004/20868=6339/&q;}]},{&q;id&q;:87334,&q;title&q;:&q;Батарея&q;,&q;order&q;:78,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;Несъемная&q;,&q;href&q;:&q;/notebooks/c80004/batareya-87334=426988/&q;}]},{&q;id&q;:69730,&q;title&q;:&q;Количество слотов для оперативной памяти&q;,&q;order&q;:84,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;1&q;,&q;href&q;:&q;/notebooks/c80004/69730=272556/&q;}]},{&q;id&q;:56017,&q;title&q;:&q;Украинская раскладка клавиатуры&q;,&q;order&q;:90,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;С&q;,&q;href&q;:&q;/notebooks/c80004/56017=146942/&q;}]},{&q;id&q;:20872,&q;title&q;:&q;Вес&q;,&q;order&q;:102,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;2.28 кг&q;,&q;href&q;:null}]},{&q;id&q;:20879,&q;title&q;:&q;Тип оперативной памяти&q;,&q;order&q;:165,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;DDR4-2666 МГц&q;,&q;href&q;:null}]},{&q;id&q;:20880,&q;title&q;:&q;Дополнительные возможности&q;,&q;order&q;:174,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;Веб-камера HD&l;br&g; \rВстроенный микрофон&l;br&g; \rВстроенные динамики&q;,&q;href&q;:null}]},{&q;id&q;:20865,&q;title&q;:&q;Графический адаптер&q;,&q;order&q;:340,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;Интегрированная, Intel UHD 600&q;,&q;href&q;:null}]},{&q;id&q;:20867,&q;title&q;:&q;Сетевые адаптеры&q;,&q;order&q;:420,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;Wi-Fi 802.11&l;br&g;\rBluetooth&q;,&q;href&q;:null}]},{&q;id&q;:20869,&q;title&q;:&q;Разъемы и порты ввода-вывода&q;,&q;order&q;:560,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;2 x USB 3.1 Gen1 / 1 x USB 2.0 / HDMI / комбинированный аудиоразъем для наушников/микрофона / кардридер&q;,&q;href&q;:null}]},{&q;id&q;:20870,&q;title&q;:&q;Характеристики батареи&q;,&q;order&q;:720,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;3-ячеечная, 42 Вт*ч&q;,&q;href&q;:null}]},{&q;id&q;:20871,&q;title&q;:&q;Габариты (Ш х Г х В)&q;,&q;order&q;:740,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;380 x 258 x 22.7 мм&q;,&q;href&q;:null}]},{&q;id&q;:98900,&q;title&q;:&q;Страна-производитель товара&q;,&q;order&q;:800,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;Китай&q;,&q;href&q;:&q;/notebooks/c80004/strana-proizvoditelj-tovara-90098=544331/&q;}]},{&q;id&q;:20873,&q;title&q;:&q;Комплект поставки&q;,&q;order&q;:820,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;Ноутбук&l;br&g; \rАдаптер питания&l;br&g; \rДокументация&q;,&q;href&q;:null}]},{&q;id&q;:87790,&q;title&q;:&q;Страна регистрации бренда&q;,&q;order&q;:950,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;США&q;,&q;href&q;:&q;/notebooks/c80004/strana-registracii-brenda-87790=429016/&q;}]},{&q;id&q;:20769,&q;title&q;:&q;Гарантия&q;,&q;order&q;:999,&q;in_short&q;:0,&q;values&q;:[{&q;title&q;:&q;12 месяцев&q;,&q;href&q;:null}]}]}]},&q;headers&q;:{&q;server&q;:[&q;nginx/1.16.1&q;],&q;date&q;:[&q;Wed, 08 Apr 2020 20:33:37 GMT&q;],&q;content-type&q;:[&q;application/json; charset=UTF-8&q;],&q;transfer-encoding&q;:[&q;chunked&q;],&q;connection&q;:[&q;keep-alive&q;],&q;keep-alive&q;:[&q;timeout=20&q;],&q;vary&q;:[&q;Accept-Encoding&q;],&q;domain&q;:[&q;product-api, product-api&q;]},&q;status&q;:200,&q;statusText&q;:&q;OK&q;,&q;url&q;:&q;https://product-api.rozetka.com.ua/v3/goods/get-characteristic?front-type=xl&a;goodsId=181862304&a;lang=ru&q;},&q;G.https://product-api.rozetka.com.ua/v3/goods/get-seo-data?front-type=xl&a;goodsId=181862304&a;lang=ru?&q;:{&q;body&q;:{&q;data&q;:{&q;anchor_links&q;:[],&q;product_tags&q;:[{&q;title&q;:&q;Дешевые&q;,&q;href&q;:&q;https://rozetka.com.ua/notebooks/c80004/111610=682110/&q;},{&q;title&q;:&q;Новые&q;,&q;href&q;:&q;https://rozetka.com.ua/notebooks/c80004/111610=682138/&q;},{&q;title&q;:&q;до 15000 гривен&q;,&q;href&q;:&q;https://rozetka.com.ua/notebooks/c80004/111610=do-15000-griven/&q;},{&q;title&q;:&q;до 5000 гривен&q;,&q;href&q;:&q;https://rozetka.com.ua/notebooks/c80004/111610=do-5000-griven/&q;},{&q;title&q;:&q;до 6000 гривен&q;,&q;href&q;:&q;https://rozetka.com.ua/notebooks/c80004/111610=do-6000-griven/&q;},{&q;title&q;:&q;до 7000 гривен&q;,&q;href&q;:&q;https://rozetka.com.ua/notebooks/c80004/111610=do-7000-griven/&q;},{&q;title&q;:&q;до 8000 гривен&q;,&q;href&q;:&q;https://rozetka.com.ua/notebooks/c80004/111610=do-8000-griven/&q;}]}},&q;headers&q;:{&q;server&q;:[&q;nginx/1.16.1&q;],&q;date&q;:[&q;Wed, 08 Apr 2020 20:33:37 GMT&q;],&q;content-type&q;:[&q;application/json; charset=UTF-8&q;],&q;transfer-encoding&q;:[&q;chunked&q;],&q;connection&q;:[&q;keep-alive&q;],&q;keep-alive&q;:[&q;timeout=20&q;],&q;vary&q;:[&q;Accept-Encoding&q;],&q;domain&q;:[&q;product-api, product-api&q;]},&q;status&q;:200,&q;statusText&q;:&q;OK&q;,&q;url&q;:&q;https://product-api.rozetka.com.ua/v3/goods/get-seo-data?front-type=xl&a;goodsId=181862304&a;lang=ru&q;},&q;G.https://product-api.rozetka.com.ua/v3/comments/get?front-type=xl&a;goods=181862304&a;page=1&a;sort=date&a;limit=10&a;lang=ru?&q;:{&q;body&q;:{&q;data&q;:{&q;visible_replies&q;:3,&q;sort&q;:{&q;date&q;:{&q;title&q;:&q;По дате&q;,&q;is_selected&q;:true,&q;name&q;:&q;date&q;,&q;href&q;:&q;/&q;},&q;helpful&q;:{&q;title&q;:&q;Самые полезные&q;,&q;is_selected&q;:false,&q;name&q;:&q;helpful&q;,&q;href&q;:&q;/?sort=helpful&q;},&q;from_buyer&q;:{&q;title&q;:&q;От купивших этот товар&q;,&q;is_selected&q;:false,&q;name&q;:&q;from_buyer&q;,&q;href&q;:&q;/?sort=from_buyer&q;},&q;with_attachments&q;:{&q;title&q;:&q;С фото и видео&q;,&q;is_selected&q;:false,&q;name&q;:&q;with_attachments&q;,&q;href&q;:&q;/?sort=with_attachments&q;}},&q;pages&q;:{&q;count&q;:1,&q;selected&q;:1},&q;record&q;:{&q;id&q;:181862304,&q;href&q;:&q;https://rozetka.com.ua/dell_i35c445dil_73b/p181862304/&q;,&q;fulltitle&q;:&q;Dell Inspiron 3582 (I35C445DIL-73B) Black&q;,&q;users_rating&q;:{&q;count_comments&q;:9},&q;pl_comment_bonus_charge&q;:0,&q;pl_comment_photo_bonus_charge&q;:0,&q;pl_comment_video_bonus_charge&q;:0,&q;disable_comments_attachments&q;:false},&q;replies&q;:{&q;45707219&q;:[{&q;id&q;:45842683,&q;usertitle&q;:&q;Сан Саныч&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Ну дык, а что люди ожидают от дешового бюджетного ноута? что можно будет вставить Core i9, 32 гига оперативы и две 1080 в sli????&l;br /&g;\nНоутбуки (тем более дешовые) ну уж точно не заточены для апгрейда, их стоит покупать только если нужна будет работа в офисных приложения вне рабочего места, или вне доступа розетки, во всех остальным случаях нужно покупать стационарный ПК&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;30 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;30&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45842683,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0421\\u0430\\u043d \\u0421\\u0430\\u043d\\u044b\\u0447\&q;,\&q;created\&q;:\&q;30 \\u043c\\u0430\\u0440\\u0442\\u0430 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u041d\\u0443 \\u0434\\u044b\\u043a, \\u0430 \\u0447\\u0442\\u043e \\u043b\\u044e\\u0434\\u0438 \\u043e\\u0436\\u0438\\u0434\\u0430\\u044e\\u0442 \\u043e\\u0442 \\u0434\\u0435\\u0448\\u043e\\u0432\\u043e\\u0433\\u043e \\u0431\\u044e\\u0434\\u0436\\u0435\\u0442\\u043d\\u043e\\u0433\\u043e \\u043d\\u043e\\u0443\\u0442\\u0430? \\u0447\\u0442\\u043e \\u043c\\u043e\\u0436\\u043d\\u043e \\u0431\\u0443\\u0434\\u0435\\u0442 \\u0432\\u0441\\u0442\\u0430\\u0432\\u0438\\u0442\\u044c Core i9, 32 \\u0433\\u0438\\u0433\\u0430 \\u043e\\u043f\\u0435\\u0440\\u0430\\u0442\\u0438\\u0432\\u044b \\u0438 \\u0434\\u0432\\u0435 1080 \\u0432 sli????&l;br \\/&g;\\n\\u041d\\u043e\\u0443\\u0442\\u0431\\u0443\\u043a\\u0438 (\\u0442\\u0435\\u043c \\u0431\\u043e\\u043b\\u0435\\u0435 \\u0434\\u0435\\u0448\\u043e\\u0432\\u044b\\u0435) \\u043d\\u0443 \\u0443\\u0436 \\u0442\\u043e\\u0447\\u043d\\u043e \\u043d\\u0435 \\u0437\\u0430\\u0442\\u043e\\u0447\\u0435\\u043d\\u044b \\u0434\\u043b\\u044f \\u0430\\u043f\\u0433\\u0440\\u0435\\u0439\\u0434\\u0430, \\u0438\\u0445 \\u0441\\u0442\\u043e\\u0438\\u0442 \\u043f\\u043e\\u043a\\u0443\\u043f\\u0430\\u0442\\u044c \\u0442\\u043e\\u043b\\u044c\\u043a\\u043e \\u0435\\u0441\\u043b\\u0438 \\u043d\\u0443\\u0436\\u043d\\u0430 \\u0431\\u0443\\u0434\\u0435\\u0442 \\u0440\\u0430\\u0431\\u043e\\u0442\\u0430 \\u0432 \\u043e\\u0444\\u0438\\u0441\\u043d\\u044b\\u0445 \\u043f\\u0440\\u0438\\u043b\\u043e\\u0436\\u0435\\u043d\\u0438\\u044f \\u0432\\u043d\\u0435 \\u0440\\u0430\\u0431\\u043e\\u0447\\u0435\\u0433\\u043e \\u043c\\u0435\\u0441\\u0442\\u0430, \\u0438\\u043b\\u0438 \\u0432\\u043d\\u0435 \\u0434\\u043e\\u0441\\u0442\\u0443\\u043f\\u0430 \\u0440\\u043e\\u0437\\u0435\\u0442\\u043a\\u0438, \\u0432\\u043e \\u0432\\u0441\\u0435\\u0445 \\u043e\\u0441\\u0442\\u0430\\u043b\\u044c\\u043d\\u044b\\u043c \\u0441\\u043b\\u0443\\u0447\\u0430\\u044f\\u0445 \\u043d\\u0443\\u0436\\u043d\\u043e \\u043f\\u043e\\u043a\\u0443\\u043f\\u0430\\u0442\\u044c \\u0441\\u0442\\u0430\\u0446\\u0438\\u043e\\u043d\\u0430\\u0440\\u043d\\u044b\\u0439 \\u041f\\u041a\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-03-30T18:43:34+03:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;45792616&q;:[{&q;id&q;:45800873,&q;usertitle&q;:&q;Юрій Нікітін&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Ні, проц слабий... відеокарти практично немає&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;26 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;26&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45800873,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u042e\\u0440\\u0456\\u0439 \\u041d\\u0456\\u043a\\u0456\\u0442\\u0456\\u043d\&q;,\&q;created\&q;:\&q;26 \\u043c\\u0430\\u0440\\u0442\\u0430 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u041d\\u0456, \\u043f\\u0440\\u043e\\u0446 \\u0441\\u043b\\u0430\\u0431\\u0438\\u0439... \\u0432\\u0456\\u0434\\u0435\\u043e\\u043a\\u0430\\u0440\\u0442\\u0438 \\u043f\\u0440\\u0430\\u043a\\u0442\\u0438\\u0447\\u043d\\u043e \\u043d\\u0435\\u043c\\u0430\\u0454\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-03-26T11:26:04+02:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;45801187&q;:[{&q;id&q;:45807637,&q;usertitle&q;:&q;Даниил&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;https://rozetka.com.ua/ua/lenovo_81d600tdra/p137439315/ эта модель намного мощнее за ту же цену. В чистое небо будет примерно  на 50 процентов ниже фпс.&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;26 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;26&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45807637,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0414\\u0430\\u043d\\u0438\\u0438\\u043b\&q;,\&q;created\&q;:\&q;26 \\u043c\\u0430\\u0440\\u0442\\u0430 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;https:\\/\\/rozetka.com.ua\\/ua\\/lenovo_81d600tdra\\/p137439315\\/ \\u044d\\u0442\\u0430 \\u043c\\u043e\\u0434\\u0435\\u043b\\u044c \\u043d\\u0430\\u043c\\u043d\\u043e\\u0433\\u043e \\u043c\\u043e\\u0449\\u043d\\u0435\\u0435 \\u0437\\u0430 \\u0442\\u0443 \\u0436\\u0435 \\u0446\\u0435\\u043d\\u0443. \\u0412 \\u0447\\u0438\\u0441\\u0442\\u043e\\u0435 \\u043d\\u0435\\u0431\\u043e \\u0431\\u0443\\u0434\\u0435\\u0442 \\u043f\\u0440\\u0438\\u043c\\u0435\\u0440\\u043d\\u043e  \\u043d\\u0430 50 \\u043f\\u0440\\u043e\\u0446\\u0435\\u043d\\u0442\\u043e\\u0432 \\u043d\\u0438\\u0436\\u0435 \\u0444\\u043f\\u0441.\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-03-26T22:02:08+02:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;},{&q;id&q;:45807745,&q;usertitle&q;:&q;Даниил&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Хотя этот вариант куда лучше https://rozetka.com.ua/ua/lenovo_81d600m0ra/p69310928/&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;26 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;26&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45807745,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0414\\u0430\\u043d\\u0438\\u0438\\u043b\&q;,\&q;created\&q;:\&q;26 \\u043c\\u0430\\u0440\\u0442\\u0430 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u0425\\u043e\\u0442\\u044f \\u044d\\u0442\\u043e\\u0442 \\u0432\\u0430\\u0440\\u0438\\u0430\\u043d\\u0442 \\u043a\\u0443\\u0434\\u0430 \\u043b\\u0443\\u0447\\u0448\\u0435 https:\\/\\/rozetka.com.ua\\/ua\\/lenovo_81d600m0ra\\/p69310928\\/\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-03-26T22:17:49+02:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;45805888&q;:[{&q;id&q;:45842626,&q;usertitle&q;:&q;Сан Саныч&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;дискретный видеоадапрет за такую цену???? Спуститесь с небес!!!&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;30 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;30&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45842626,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0421\\u0430\\u043d \\u0421\\u0430\\u043d\\u044b\\u0447\&q;,\&q;created\&q;:\&q;30 \\u043c\\u0430\\u0440\\u0442\\u0430 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u0434\\u0438\\u0441\\u043a\\u0440\\u0435\\u0442\\u043d\\u044b\\u0439 \\u0432\\u0438\\u0434\\u0435\\u043e\\u0430\\u0434\\u0430\\u043f\\u0440\\u0435\\u0442 \\u0437\\u0430 \\u0442\\u0430\\u043a\\u0443\\u044e \\u0446\\u0435\\u043d\\u0443???? \\u0421\\u043f\\u0443\\u0441\\u0442\\u0438\\u0442\\u0435\\u0441\\u044c \\u0441 \\u043d\\u0435\\u0431\\u0435\\u0441!!!\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-03-30T18:38:59+03:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;45875237&q;:[{&q;id&q;:45915439,&q;usertitle&q;:&q;Артур&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Скільки думав про індикатор роботи диску на ноутбуках і все ж не можу зрозуміти - для чого вони взагалі потрібні і що в них корисного.&l;br /&g;\nНе підкажете, чому це так важливо для вас?&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;6 апреля 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:4,&q;day&q;:&q;06&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45915439,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0410\\u0440\\u0442\\u0443\\u0440\&q;,\&q;created\&q;:\&q;6 \\u0430\\u043f\\u0440\\u0435\\u043b\\u044f 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u0421\\u043a\\u0456\\u043b\\u044c\\u043a\\u0438 \\u0434\\u0443\\u043c\\u0430\\u0432 \\u043f\\u0440\\u043e \\u0456\\u043d\\u0434\\u0438\\u043a\\u0430\\u0442\\u043e\\u0440 \\u0440\\u043e\\u0431\\u043e\\u0442\\u0438 \\u0434\\u0438\\u0441\\u043a\\u0443 \\u043d\\u0430 \\u043d\\u043e\\u0443\\u0442\\u0431\\u0443\\u043a\\u0430\\u0445 \\u0456 \\u0432\\u0441\\u0435 \\u0436 \\u043d\\u0435 \\u043c\\u043e\\u0436\\u0443 \\u0437\\u0440\\u043e\\u0437\\u0443\\u043c\\u0456\\u0442\\u0438 - \\u0434\\u043b\\u044f \\u0447\\u043e\\u0433\\u043e \\u0432\\u043e\\u043d\\u0438 \\u0432\\u0437\\u0430\\u0433\\u0430\\u043b\\u0456 \\u043f\\u043e\\u0442\\u0440\\u0456\\u0431\\u043d\\u0456 \\u0456 \\u0449\\u043e \\u0432 \\u043d\\u0438\\u0445 \\u043a\\u043e\\u0440\\u0438\\u0441\\u043d\\u043e\\u0433\\u043e.&l;br \\/&g;\\n\\u041d\\u0435 \\u043f\\u0456\\u0434\\u043a\\u0430\\u0436\\u0435\\u0442\\u0435, \\u0447\\u043e\\u043c\\u0443 \\u0446\\u0435 \\u0442\\u0430\\u043a \\u0432\\u0430\\u0436\\u043b\\u0438\\u0432\\u043e \\u0434\\u043b\\u044f \\u0432\\u0430\\u0441?\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-04-06T23:35:21+03:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;},{&q;id&q;:45930273,&q;usertitle&q;:&q;Сан Саныч&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Артур, немає значення, ноутбук чи стаціонарный ПК, функція індикатора одна й та сама - якщо він активно світиться, значить в цей час якась програма або операційна система звертається до жорсткого диску і тому не бажано запускати якусь іншу програму, оскільки в цьому випадку буде все гальмувати. По-моєму це очевидно&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;сегодня&q;,&q;year&q;:&q;2020&q;,&q;month&q;:4,&q;day&q;:&q;08&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45930273,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0421\\u0430\\u043d \\u0421\\u0430\\u043d\\u044b\\u0447\&q;,\&q;created\&q;:\&q;\\u0441\\u0435\\u0433\\u043e\\u0434\\u043d\\u044f\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u0410\\u0440\\u0442\\u0443\\u0440, \\u043d\\u0435\\u043c\\u0430\\u0454 \\u0437\\u043d\\u0430\\u0447\\u0435\\u043d\\u043d\\u044f, \\u043d\\u043e\\u0443\\u0442\\u0431\\u0443\\u043a \\u0447\\u0438 \\u0441\\u0442\\u0430\\u0446\\u0456\\u043e\\u043d\\u0430\\u0440\\u043d\\u044b\\u0439 \\u041f\\u041a, \\u0444\\u0443\\u043d\\u043a\\u0446\\u0456\\u044f \\u0456\\u043d\\u0434\\u0438\\u043a\\u0430\\u0442\\u043e\\u0440\\u0430 \\u043e\\u0434\\u043d\\u0430 \\u0439 \\u0442\\u0430 \\u0441\\u0430\\u043c\\u0430 - \\u044f\\u043a\\u0449\\u043e \\u0432\\u0456\\u043d \\u0430\\u043a\\u0442\\u0438\\u0432\\u043d\\u043e \\u0441\\u0432\\u0456\\u0442\\u0438\\u0442\\u044c\\u0441\\u044f, \\u0437\\u043d\\u0430\\u0447\\u0438\\u0442\\u044c \\u0432 \\u0446\\u0435\\u0439 \\u0447\\u0430\\u0441 \\u044f\\u043a\\u0430\\u0441\\u044c \\u043f\\u0440\\u043e\\u0433\\u0440\\u0430\\u043c\\u0430 \\u0430\\u0431\\u043e \\u043e\\u043f\\u0435\\u0440\\u0430\\u0446\\u0456\\u0439\\u043d\\u0430 \\u0441\\u0438\\u0441\\u0442\\u0435\\u043c\\u0430 \\u0437\\u0432\\u0435\\u0440\\u0442\\u0430\\u0454\\u0442\\u044c\\u0441\\u044f \\u0434\\u043e \\u0436\\u043e\\u0440\\u0441\\u0442\\u043a\\u043e\\u0433\\u043e \\u0434\\u0438\\u0441\\u043a\\u0443 \\u0456 \\u0442\\u043e\\u043c\\u0443 \\u043d\\u0435 \\u0431\\u0430\\u0436\\u0430\\u043d\\u043e \\u0437\\u0430\\u043f\\u0443\\u0441\\u043a\\u0430\\u0442\\u0438 \\u044f\\u043a\\u0443\\u0441\\u044c \\u0456\\u043d\\u0448\\u0443 \\u043f\\u0440\\u043e\\u0433\\u0440\\u0430\\u043c\\u0443, \\u043e\\u0441\\u043a\\u0456\\u043b\\u044c\\u043a\\u0438 \\u0432 \\u0446\\u044c\\u043e\\u043c\\u0443 \\u0432\\u0438\\u043f\\u0430\\u0434\\u043a\\u0443 \\u0431\\u0443\\u0434\\u0435 \\u0432\\u0441\\u0435 \\u0433\\u0430\\u043b\\u044c\\u043c\\u0443\\u0432\\u0430\\u0442\\u0438. \\u041f\\u043e-\\u043c\\u043e\\u0454\\u043c\\u0443 \\u0446\\u0435 \\u043e\\u0447\\u0435\\u0432\\u0438\\u0434\\u043d\\u043e\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-04-08T12:07:10+03:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;45902573&q;:[{&q;id&q;:45903995,&q;usertitle&q;:&q;Андрей Смирнов&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Нет. Винду надо самому ставить&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;5 апреля 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:4,&q;day&q;:&q;05&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45903995,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0410\\u043d\\u0434\\u0440\\u0435\\u0439 \\u0421\\u043c\\u0438\\u0440\\u043d\\u043e\\u0432\&q;,\&q;created\&q;:\&q;5 \\u0430\\u043f\\u0440\\u0435\\u043b\\u044f 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u041d\\u0435\\u0442. \\u0412\\u0438\\u043d\\u0434\\u0443 \\u043d\\u0430\\u0434\\u043e \\u0441\\u0430\\u043c\\u043e\\u043c\\u0443 \\u0441\\u0442\\u0430\\u0432\\u0438\\u0442\\u044c\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-04-05T21:38:53+03:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}]},&q;comments&q;:[{&q;id&q;:45902573,&q;usertitle&q;:&q;Тоб Роб&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;14&q;,&q;text&q;:&q;А он на операцоной систем віндеус?&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;5 апреля 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:4,&q;day&q;:&q;05&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:[],&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:{&q;id&q;:45902573,&q;goods_id&q;:&q;181862304&q;,&q;user_title&q;:&q;Тоб Роб&q;,&q;created&q;:&q;5 апреля 2020&q;,&q;from_buyer&q;:false,&q;mark&q;:null,&q;text&q;:&q;А он на операцоной систем віндеус?&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;positive&q;:&q;1&q;,&q;negative&q;:&q;6&q;,&q;replies&q;:[{&q;created&q;:&q;5 апреля 2020&q;,&q;user_title&q;:&q;Андрей Смирнов&q;,&q;text&q;:&q;Нет. Винду надо самому ставить&q;}],&q;marks&q;:[],&q;questions&q;:[],&q;created_iso&q;:&q;2020-04-05T18:18:55+03:00&q;},&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;,&q;seller_id&q;:5,&q;replies&q;:[{&q;id&q;:45903995,&q;usertitle&q;:&q;Андрей Смирнов&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Нет. Винду надо самому ставить&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;5 апреля 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:4,&q;day&q;:&q;05&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45903995,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0410\\u043d\\u0434\\u0440\\u0435\\u0439 \\u0421\\u043c\\u0438\\u0440\\u043d\\u043e\\u0432\&q;,\&q;created\&q;:\&q;5 \\u0430\\u043f\\u0440\\u0435\\u043b\\u044f 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u041d\\u0435\\u0442. \\u0412\\u0438\\u043d\\u0434\\u0443 \\u043d\\u0430\\u0434\\u043e \\u0441\\u0430\\u043c\\u043e\\u043c\\u0443 \\u0441\\u0442\\u0430\\u0432\\u0438\\u0442\\u044c\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-04-05T21:38:53+03:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;votes&q;:{&q;positive&q;:1,&q;negative&q;:6,&q;total&q;:7}},{&q;id&q;:45875237,&q;usertitle&q;:&q;Святослав&q;,&q;mark&q;:0,&q;percent_dignity&q;:&q;100&q;,&q;text&q;:&q;Добрий варіант на цю ціну.&q;,&q;dignity&q;:&q;Легкий, тоненький, елегантний. Достатньо шкидкий  - для інтернет серфінгу, скайпу, офісної роботи, фільмів та он-лайн ТБ&q;,&q;shortcomings&q;:&q;Немає індикації роботи диска, зарядки окремими  led діодами&q;,&q;from_buyer&q;:true,&q;created&q;:{&q;pop_date&q;:&q;2 апреля 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:4,&q;day&q;:&q;02&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:[],&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:{&q;id&q;:45875237,&q;goods_id&q;:&q;181862304&q;,&q;user_title&q;:&q;Святослав&q;,&q;created&q;:&q;2 апреля 2020&q;,&q;from_buyer&q;:true,&q;mark&q;:0,&q;text&q;:&q;Добрий варіант на цю ціну.&q;,&q;dignity&q;:&q;Легкий, тоненький, елегантний. Достатньо шкидкий  - для інтернет серфінгу, скайпу, офісної роботи, фільмів та он-лайн ТБ&q;,&q;shortcomings&q;:&q;Немає індикації роботи диска, зарядки окремими  led діодами&q;,&q;positive&q;:&q;2&q;,&q;negative&q;:&q;0&q;,&q;replies&q;:[{&q;created&q;:&q;6 апреля 2020&q;,&q;user_title&q;:&q;Артур&q;,&q;text&q;:&q;Скільки думав про індикатор роботи диску на ноутбуках і все ж не можу зрозуміти - для чого вони взагалі потрібні і що в них корисного.&l;br /&g;\nНе підкажете, чому це так важливо для вас?&q;},{&q;created&q;:&q;сегодня&q;,&q;user_title&q;:&q;Сан Саныч&q;,&q;text&q;:&q;Артур, немає значення, ноутбук чи стаціонарный ПК, функція індикатора одна й та сама - якщо він активно світиться, значить в цей час якась програма або операційна система звертається до жорсткого диску і тому не бажано запускати якусь іншу програму, оскільки в цьому випадку буде все гальмувати. По-моєму це очевидно&q;}],&q;marks&q;:[],&q;questions&q;:[],&q;created_iso&q;:&q;2020-04-02T19:33:40+03:00&q;},&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;,&q;seller_id&q;:5,&q;replies&q;:[{&q;id&q;:45915439,&q;usertitle&q;:&q;Артур&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Скільки думав про індикатор роботи диску на ноутбуках і все ж не можу зрозуміти - для чого вони взагалі потрібні і що в них корисного.&l;br /&g;\nНе підкажете, чому це так важливо для вас?&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;6 апреля 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:4,&q;day&q;:&q;06&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45915439,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0410\\u0440\\u0442\\u0443\\u0440\&q;,\&q;created\&q;:\&q;6 \\u0430\\u043f\\u0440\\u0435\\u043b\\u044f 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u0421\\u043a\\u0456\\u043b\\u044c\\u043a\\u0438 \\u0434\\u0443\\u043c\\u0430\\u0432 \\u043f\\u0440\\u043e \\u0456\\u043d\\u0434\\u0438\\u043a\\u0430\\u0442\\u043e\\u0440 \\u0440\\u043e\\u0431\\u043e\\u0442\\u0438 \\u0434\\u0438\\u0441\\u043a\\u0443 \\u043d\\u0430 \\u043d\\u043e\\u0443\\u0442\\u0431\\u0443\\u043a\\u0430\\u0445 \\u0456 \\u0432\\u0441\\u0435 \\u0436 \\u043d\\u0435 \\u043c\\u043e\\u0436\\u0443 \\u0437\\u0440\\u043e\\u0437\\u0443\\u043c\\u0456\\u0442\\u0438 - \\u0434\\u043b\\u044f \\u0447\\u043e\\u0433\\u043e \\u0432\\u043e\\u043d\\u0438 \\u0432\\u0437\\u0430\\u0433\\u0430\\u043b\\u0456 \\u043f\\u043e\\u0442\\u0440\\u0456\\u0431\\u043d\\u0456 \\u0456 \\u0449\\u043e \\u0432 \\u043d\\u0438\\u0445 \\u043a\\u043e\\u0440\\u0438\\u0441\\u043d\\u043e\\u0433\\u043e.&l;br \\/&g;\\n\\u041d\\u0435 \\u043f\\u0456\\u0434\\u043a\\u0430\\u0436\\u0435\\u0442\\u0435, \\u0447\\u043e\\u043c\\u0443 \\u0446\\u0435 \\u0442\\u0430\\u043a \\u0432\\u0430\\u0436\\u043b\\u0438\\u0432\\u043e \\u0434\\u043b\\u044f \\u0432\\u0430\\u0441?\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-04-06T23:35:21+03:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;},{&q;id&q;:45930273,&q;usertitle&q;:&q;Сан Саныч&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Артур, немає значення, ноутбук чи стаціонарный ПК, функція індикатора одна й та сама - якщо він активно світиться, значить в цей час якась програма або операційна система звертається до жорсткого диску і тому не бажано запускати якусь іншу програму, оскільки в цьому випадку буде все гальмувати. По-моєму це очевидно&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;сегодня&q;,&q;year&q;:&q;2020&q;,&q;month&q;:4,&q;day&q;:&q;08&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45930273,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0421\\u0430\\u043d \\u0421\\u0430\\u043d\\u044b\\u0447\&q;,\&q;created\&q;:\&q;\\u0441\\u0435\\u0433\\u043e\\u0434\\u043d\\u044f\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u0410\\u0440\\u0442\\u0443\\u0440, \\u043d\\u0435\\u043c\\u0430\\u0454 \\u0437\\u043d\\u0430\\u0447\\u0435\\u043d\\u043d\\u044f, \\u043d\\u043e\\u0443\\u0442\\u0431\\u0443\\u043a \\u0447\\u0438 \\u0441\\u0442\\u0430\\u0446\\u0456\\u043e\\u043d\\u0430\\u0440\\u043d\\u044b\\u0439 \\u041f\\u041a, \\u0444\\u0443\\u043d\\u043a\\u0446\\u0456\\u044f \\u0456\\u043d\\u0434\\u0438\\u043a\\u0430\\u0442\\u043e\\u0440\\u0430 \\u043e\\u0434\\u043d\\u0430 \\u0439 \\u0442\\u0430 \\u0441\\u0430\\u043c\\u0430 - \\u044f\\u043a\\u0449\\u043e \\u0432\\u0456\\u043d \\u0430\\u043a\\u0442\\u0438\\u0432\\u043d\\u043e \\u0441\\u0432\\u0456\\u0442\\u0438\\u0442\\u044c\\u0441\\u044f, \\u0437\\u043d\\u0430\\u0447\\u0438\\u0442\\u044c \\u0432 \\u0446\\u0435\\u0439 \\u0447\\u0430\\u0441 \\u044f\\u043a\\u0430\\u0441\\u044c \\u043f\\u0440\\u043e\\u0433\\u0440\\u0430\\u043c\\u0430 \\u0430\\u0431\\u043e \\u043e\\u043f\\u0435\\u0440\\u0430\\u0446\\u0456\\u0439\\u043d\\u0430 \\u0441\\u0438\\u0441\\u0442\\u0435\\u043c\\u0430 \\u0437\\u0432\\u0435\\u0440\\u0442\\u0430\\u0454\\u0442\\u044c\\u0441\\u044f \\u0434\\u043e \\u0436\\u043e\\u0440\\u0441\\u0442\\u043a\\u043e\\u0433\\u043e \\u0434\\u0438\\u0441\\u043a\\u0443 \\u0456 \\u0442\\u043e\\u043c\\u0443 \\u043d\\u0435 \\u0431\\u0430\\u0436\\u0430\\u043d\\u043e \\u0437\\u0430\\u043f\\u0443\\u0441\\u043a\\u0430\\u0442\\u0438 \\u044f\\u043a\\u0443\\u0441\\u044c \\u0456\\u043d\\u0448\\u0443 \\u043f\\u0440\\u043e\\u0433\\u0440\\u0430\\u043c\\u0443, \\u043e\\u0441\\u043a\\u0456\\u043b\\u044c\\u043a\\u0438 \\u0432 \\u0446\\u044c\\u043e\\u043c\\u0443 \\u0432\\u0438\\u043f\\u0430\\u0434\\u043a\\u0443 \\u0431\\u0443\\u0434\\u0435 \\u0432\\u0441\\u0435 \\u0433\\u0430\\u043b\\u044c\\u043c\\u0443\\u0432\\u0430\\u0442\\u0438. \\u041f\\u043e-\\u043c\\u043e\\u0454\\u043c\\u0443 \\u0446\\u0435 \\u043e\\u0447\\u0435\\u0432\\u0438\\u0434\\u043d\\u043e\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-04-08T12:07:10+03:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;mark_percent&q;:&q;0%&q;,&q;votes&q;:{&q;positive&q;:2,&q;negative&q;:0,&q;total&q;:2}},{&q;id&q;:45860086,&q;usertitle&q;:&q;UNDERGROUND&q;,&q;mark&q;:3,&q;percent_dignity&q;:&q;100&q;,&q;text&q;:&q;50/50&q;,&q;dignity&q;:&q;Сравнительно тихий из-за отсутствия кулеров. Сборка неплохая. Нормальный тачпад.&q;,&q;shortcomings&q;:&q;Процессор 1.1ггц это аут. На рабочем столе грузится на 100%&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;1 апреля 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:4,&q;day&q;:&q;01&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:[],&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:{&q;id&q;:45860086,&q;goods_id&q;:&q;181862304&q;,&q;user_title&q;:&q;UNDERGROUND&q;,&q;created&q;:&q;1 апреля 2020&q;,&q;from_buyer&q;:false,&q;mark&q;:3,&q;text&q;:&q;50/50&q;,&q;dignity&q;:&q;Сравнительно тихий из-за отсутствия кулеров. Сборка неплохая. Нормальный тачпад.&q;,&q;shortcomings&q;:&q;Процессор 1.1ггц это аут. На рабочем столе грузится на 100%&q;,&q;positive&q;:&q;1&q;,&q;negative&q;:&q;0&q;,&q;replies&q;:[],&q;marks&q;:[],&q;questions&q;:[],&q;created_iso&q;:&q;2020-04-01T13:46:42+03:00&q;},&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;,&q;seller_id&q;:5,&q;replies&q;:[],&q;mark_percent&q;:&q;60%&q;,&q;votes&q;:{&q;positive&q;:1,&q;negative&q;:0,&q;total&q;:1}},{&q;id&q;:45842363,&q;usertitle&q;:&q;Алексей&q;,&q;mark&q;:4,&q;percent_dignity&q;:&q;100&q;,&q;text&q;:&q;Первую неделю работал - обычные офисные задачи, браузер - очень туго. Потом поменял оперативку на 8ГБ - небо и земля. Хороший ноут на свои деньги. Качество сборки понравилось.&q;,&q;dignity&q;:&q;сборка, цена, вес, минималистический дизайн&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;30 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;30&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:[],&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:{&q;id&q;:45842363,&q;goods_id&q;:&q;181862304&q;,&q;user_title&q;:&q;Алексей&q;,&q;created&q;:&q;30 марта 2020&q;,&q;from_buyer&q;:false,&q;mark&q;:4,&q;text&q;:&q;Первую неделю работал - обычные офисные задачи, браузер - очень туго. Потом поменял оперативку на 8ГБ - небо и земля. Хороший ноут на свои деньги. Качество сборки понравилось.&q;,&q;dignity&q;:&q;сборка, цена, вес, минималистический дизайн&q;,&q;shortcomings&q;:&q;&q;,&q;positive&q;:&q;1&q;,&q;negative&q;:&q;0&q;,&q;replies&q;:[],&q;marks&q;:[],&q;questions&q;:[],&q;created_iso&q;:&q;2020-03-30T18:15:00+03:00&q;},&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;,&q;seller_id&q;:5,&q;replies&q;:[],&q;mark_percent&q;:&q;80%&q;,&q;votes&q;:{&q;positive&q;:1,&q;negative&q;:0,&q;total&q;:1}},{&q;id&q;:45805888,&q;usertitle&q;:&q;Саша Василенко&q;,&q;mark&q;:3,&q;percent_dignity&q;:&q;100&q;,&q;text&q;:&q;По этой цене можно смело брать для офисной работы&q;,&q;dignity&q;:&q;Имеется ДВД,вместо которого можно вставить карман с родным HDD, а вместо HDD поставитьSSD&q;,&q;shortcomings&q;:&q;нет дискретного видеоадаптера&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;26 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;26&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:[],&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:{&q;id&q;:45805888,&q;goods_id&q;:&q;181862304&q;,&q;user_title&q;:&q;Саша Василенко&q;,&q;created&q;:&q;26 марта 2020&q;,&q;from_buyer&q;:false,&q;mark&q;:3,&q;text&q;:&q;По этой цене можно смело брать для офисной работы&q;,&q;dignity&q;:&q;Имеется ДВД,вместо которого можно вставить карман с родным HDD, а вместо HDD поставитьSSD&q;,&q;shortcomings&q;:&q;нет дискретного видеоадаптера&q;,&q;positive&q;:&q;2&q;,&q;negative&q;:&q;0&q;,&q;replies&q;:[{&q;created&q;:&q;30 марта 2020&q;,&q;user_title&q;:&q;Сан Саныч&q;,&q;text&q;:&q;дискретный видеоадапрет за такую цену???? Спуститесь с небес!!!&q;}],&q;marks&q;:[],&q;questions&q;:[],&q;created_iso&q;:&q;2020-03-26T18:59:53+02:00&q;},&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;,&q;seller_id&q;:5,&q;replies&q;:[{&q;id&q;:45842626,&q;usertitle&q;:&q;Сан Саныч&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;дискретный видеоадапрет за такую цену???? Спуститесь с небес!!!&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;30 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;30&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45842626,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0421\\u0430\\u043d \\u0421\\u0430\\u043d\\u044b\\u0447\&q;,\&q;created\&q;:\&q;30 \\u043c\\u0430\\u0440\\u0442\\u0430 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u0434\\u0438\\u0441\\u043a\\u0440\\u0435\\u0442\\u043d\\u044b\\u0439 \\u0432\\u0438\\u0434\\u0435\\u043e\\u0430\\u0434\\u0430\\u043f\\u0440\\u0435\\u0442 \\u0437\\u0430 \\u0442\\u0430\\u043a\\u0443\\u044e \\u0446\\u0435\\u043d\\u0443???? \\u0421\\u043f\\u0443\\u0441\\u0442\\u0438\\u0442\\u0435\\u0441\\u044c \\u0441 \\u043d\\u0435\\u0431\\u0435\\u0441!!!\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-03-30T18:38:59+03:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;mark_percent&q;:&q;60%&q;,&q;votes&q;:{&q;positive&q;:2,&q;negative&q;:0,&q;total&q;:2}},{&q;id&q;:45801187,&q;usertitle&q;:&q;Меня Зовут&q;,&q;mark&q;:0,&q;percent_dignity&q;:&q;100&q;,&q;text&q;:&q;А сталкер чистое небо потянет хоть на минималка?&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;26 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;26&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:[],&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:{&q;id&q;:45801187,&q;goods_id&q;:&q;181862304&q;,&q;user_title&q;:&q;Меня Зовут&q;,&q;created&q;:&q;26 марта 2020&q;,&q;from_buyer&q;:false,&q;mark&q;:0,&q;text&q;:&q;А сталкер чистое небо потянет хоть на минималка?&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;positive&q;:&q;1&q;,&q;negative&q;:&q;0&q;,&q;replies&q;:[{&q;created&q;:&q;26 марта 2020&q;,&q;user_title&q;:&q;Даниил&q;,&q;text&q;:&q;https://rozetka.com.ua/ua/lenovo_81d600tdra/p137439315/ эта модель намного мощнее за ту же цену. В чистое небо будет примерно  на 50 процентов ниже фпс.&q;},{&q;created&q;:&q;26 марта 2020&q;,&q;user_title&q;:&q;Даниил&q;,&q;text&q;:&q;Хотя этот вариант куда лучше https://rozetka.com.ua/ua/lenovo_81d600m0ra/p69310928/&q;}],&q;marks&q;:[],&q;questions&q;:[],&q;created_iso&q;:&q;2020-03-26T11:56:14+02:00&q;},&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;,&q;seller_id&q;:5,&q;replies&q;:[{&q;id&q;:45807637,&q;usertitle&q;:&q;Даниил&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;https://rozetka.com.ua/ua/lenovo_81d600tdra/p137439315/ эта модель намного мощнее за ту же цену. В чистое небо будет примерно  на 50 процентов ниже фпс.&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;26 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;26&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45807637,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0414\\u0430\\u043d\\u0438\\u0438\\u043b\&q;,\&q;created\&q;:\&q;26 \\u043c\\u0430\\u0440\\u0442\\u0430 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;https:\\/\\/rozetka.com.ua\\/ua\\/lenovo_81d600tdra\\/p137439315\\/ \\u044d\\u0442\\u0430 \\u043c\\u043e\\u0434\\u0435\\u043b\\u044c \\u043d\\u0430\\u043c\\u043d\\u043e\\u0433\\u043e \\u043c\\u043e\\u0449\\u043d\\u0435\\u0435 \\u0437\\u0430 \\u0442\\u0443 \\u0436\\u0435 \\u0446\\u0435\\u043d\\u0443. \\u0412 \\u0447\\u0438\\u0441\\u0442\\u043e\\u0435 \\u043d\\u0435\\u0431\\u043e \\u0431\\u0443\\u0434\\u0435\\u0442 \\u043f\\u0440\\u0438\\u043c\\u0435\\u0440\\u043d\\u043e  \\u043d\\u0430 50 \\u043f\\u0440\\u043e\\u0446\\u0435\\u043d\\u0442\\u043e\\u0432 \\u043d\\u0438\\u0436\\u0435 \\u0444\\u043f\\u0441.\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-03-26T22:02:08+02:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;},{&q;id&q;:45807745,&q;usertitle&q;:&q;Даниил&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Хотя этот вариант куда лучше https://rozetka.com.ua/ua/lenovo_81d600m0ra/p69310928/&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;26 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;26&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45807745,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0414\\u0430\\u043d\\u0438\\u0438\\u043b\&q;,\&q;created\&q;:\&q;26 \\u043c\\u0430\\u0440\\u0442\\u0430 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u0425\\u043e\\u0442\\u044f \\u044d\\u0442\\u043e\\u0442 \\u0432\\u0430\\u0440\\u0438\\u0430\\u043d\\u0442 \\u043a\\u0443\\u0434\\u0430 \\u043b\\u0443\\u0447\\u0448\\u0435 https:\\/\\/rozetka.com.ua\\/ua\\/lenovo_81d600m0ra\\/p69310928\\/\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-03-26T22:17:49+02:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;mark_percent&q;:&q;0%&q;,&q;votes&q;:{&q;positive&q;:1,&q;negative&q;:0,&q;total&q;:1}},{&q;id&q;:45792616,&q;usertitle&q;:&q;Danylo Boyar&q;,&q;mark&q;:0,&q;percent_dignity&q;:&q;100&q;,&q;text&q;:&q;А World of tanks потягне?&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;25 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;25&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:[],&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:{&q;id&q;:45792616,&q;goods_id&q;:&q;181862304&q;,&q;user_title&q;:&q;Danylo Boyar&q;,&q;created&q;:&q;25 марта 2020&q;,&q;from_buyer&q;:false,&q;mark&q;:0,&q;text&q;:&q;А World of tanks потягне?&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;positive&q;:&q;3&q;,&q;negative&q;:&q;0&q;,&q;replies&q;:[{&q;created&q;:&q;26 марта 2020&q;,&q;user_title&q;:&q;Юрій Нікітін&q;,&q;text&q;:&q;Ні, проц слабий... відеокарти практично немає&q;}],&q;marks&q;:[],&q;questions&q;:[],&q;created_iso&q;:&q;2020-03-25T14:13:44+02:00&q;},&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;,&q;seller_id&q;:5,&q;replies&q;:[{&q;id&q;:45800873,&q;usertitle&q;:&q;Юрій Нікітін&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Ні, проц слабий... відеокарти практично немає&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;26 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;26&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45800873,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u042e\\u0440\\u0456\\u0439 \\u041d\\u0456\\u043a\\u0456\\u0442\\u0456\\u043d\&q;,\&q;created\&q;:\&q;26 \\u043c\\u0430\\u0440\\u0442\\u0430 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u041d\\u0456, \\u043f\\u0440\\u043e\\u0446 \\u0441\\u043b\\u0430\\u0431\\u0438\\u0439... \\u0432\\u0456\\u0434\\u0435\\u043e\\u043a\\u0430\\u0440\\u0442\\u0438 \\u043f\\u0440\\u0430\\u043a\\u0442\\u0438\\u0447\\u043d\\u043e \\u043d\\u0435\\u043c\\u0430\\u0454\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-03-26T11:26:04+02:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;mark_percent&q;:&q;0%&q;,&q;votes&q;:{&q;positive&q;:3,&q;negative&q;:0,&q;total&q;:3}},{&q;id&q;:45707219,&q;usertitle&q;:&q;Семён Разжаловец&q;,&q;mark&q;:3,&q;percent_dignity&q;:&q;40&q;,&q;text&q;:&q;Ко мне часто обращаются с просьбой усовершенствовать подобные ноутбуке. Ситуация, когда человек покупает такой аппарат для работы, думая, что дешевый ноут потянет работу в интернете и офис. Но со временем приходят обновления на систему, google chrome - 1гб оперативной памяти, скайп - ещё 1гб, офис и другие рабочие программы - вот и вся оперативка съедена.&q;,&q;dignity&q;:&q;4гб оперативной памяти&q;,&q;shortcomings&q;:&q;Слабенький ноут, практически невозможно усовершенствовать&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;15 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;15&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:[],&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:{&q;id&q;:45707219,&q;goods_id&q;:&q;181862304&q;,&q;user_title&q;:&q;Семён Разжаловец&q;,&q;created&q;:&q;15 марта 2020&q;,&q;from_buyer&q;:false,&q;mark&q;:3,&q;text&q;:&q;Ко мне часто обращаются с просьбой усовершенствовать подобные ноутбуке. Ситуация, когда человек покупает такой аппарат для работы, думая, что дешевый ноут потянет работу в интернете и офис. Но со временем приходят обновления на систему, google chrome - 1гб оперативной памяти, скайп - ещё 1гб, офис и другие рабочие программы - вот и вся оперативка съедена.&q;,&q;dignity&q;:&q;4гб оперативной памяти&q;,&q;shortcomings&q;:&q;Слабенький ноут, практически невозможно усовершенствовать&q;,&q;positive&q;:&q;2&q;,&q;negative&q;:&q;3&q;,&q;replies&q;:[{&q;created&q;:&q;30 марта 2020&q;,&q;user_title&q;:&q;Сан Саныч&q;,&q;text&q;:&q;Ну дык, а что люди ожидают от дешового бюджетного ноута? что можно будет вставить Core i9, 32 гига оперативы и две 1080 в sli????&l;br /&g;\nНоутбуки (тем более дешовые) ну уж точно не заточены для апгрейда, их стоит покупать только если нужна будет работа в офисных приложения вне рабочего места, или вне доступа розетки, во всех остальным случаях нужно покупать стационарный ПК&q;}],&q;marks&q;:[],&q;questions&q;:[],&q;created_iso&q;:&q;2020-03-15T00:13:59+02:00&q;},&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;,&q;seller_id&q;:5,&q;replies&q;:[{&q;id&q;:45842683,&q;usertitle&q;:&q;Сан Саныч&q;,&q;mark&q;:null,&q;percent_dignity&q;:&q;0&q;,&q;count_voices&q;:0,&q;count_positive_voices&q;:0,&q;negative_vote_count&q;:0,&q;text&q;:&q;Ну дык, а что люди ожидают от дешового бюджетного ноута? что можно будет вставить Core i9, 32 гига оперативы и две 1080 в sli????&l;br /&g;\nНоутбуки (тем более дешовые) ну уж точно не заточены для апгрейда, их стоит покупать только если нужна будет работа в офисных приложения вне рабочего места, или вне доступа розетки, во всех остальным случаях нужно покупать стационарный ПК&q;,&q;dignity&q;:&q;&q;,&q;shortcomings&q;:&q;&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;30 марта 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:3,&q;day&q;:&q;30&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:{&q;images&q;:[],&q;video&q;:[]},&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:&q;{\&q;id\&q;:45842683,\&q;goods_id\&q;:\&q;181862304\&q;,\&q;user_title\&q;:\&q;\\u0421\\u0430\\u043d \\u0421\\u0430\\u043d\\u044b\\u0447\&q;,\&q;created\&q;:\&q;30 \\u043c\\u0430\\u0440\\u0442\\u0430 2020\&q;,\&q;from_buyer\&q;:false,\&q;mark\&q;:null,\&q;text\&q;:\&q;\\u041d\\u0443 \\u0434\\u044b\\u043a, \\u0430 \\u0447\\u0442\\u043e \\u043b\\u044e\\u0434\\u0438 \\u043e\\u0436\\u0438\\u0434\\u0430\\u044e\\u0442 \\u043e\\u0442 \\u0434\\u0435\\u0448\\u043e\\u0432\\u043e\\u0433\\u043e \\u0431\\u044e\\u0434\\u0436\\u0435\\u0442\\u043d\\u043e\\u0433\\u043e \\u043d\\u043e\\u0443\\u0442\\u0430? \\u0447\\u0442\\u043e \\u043c\\u043e\\u0436\\u043d\\u043e \\u0431\\u0443\\u0434\\u0435\\u0442 \\u0432\\u0441\\u0442\\u0430\\u0432\\u0438\\u0442\\u044c Core i9, 32 \\u0433\\u0438\\u0433\\u0430 \\u043e\\u043f\\u0435\\u0440\\u0430\\u0442\\u0438\\u0432\\u044b \\u0438 \\u0434\\u0432\\u0435 1080 \\u0432 sli????&l;br \\/&g;\\n\\u041d\\u043e\\u0443\\u0442\\u0431\\u0443\\u043a\\u0438 (\\u0442\\u0435\\u043c \\u0431\\u043e\\u043b\\u0435\\u0435 \\u0434\\u0435\\u0448\\u043e\\u0432\\u044b\\u0435) \\u043d\\u0443 \\u0443\\u0436 \\u0442\\u043e\\u0447\\u043d\\u043e \\u043d\\u0435 \\u0437\\u0430\\u0442\\u043e\\u0447\\u0435\\u043d\\u044b \\u0434\\u043b\\u044f \\u0430\\u043f\\u0433\\u0440\\u0435\\u0439\\u0434\\u0430, \\u0438\\u0445 \\u0441\\u0442\\u043e\\u0438\\u0442 \\u043f\\u043e\\u043a\\u0443\\u043f\\u0430\\u0442\\u044c \\u0442\\u043e\\u043b\\u044c\\u043a\\u043e \\u0435\\u0441\\u043b\\u0438 \\u043d\\u0443\\u0436\\u043d\\u0430 \\u0431\\u0443\\u0434\\u0435\\u0442 \\u0440\\u0430\\u0431\\u043e\\u0442\\u0430 \\u0432 \\u043e\\u0444\\u0438\\u0441\\u043d\\u044b\\u0445 \\u043f\\u0440\\u0438\\u043b\\u043e\\u0436\\u0435\\u043d\\u0438\\u044f \\u0432\\u043d\\u0435 \\u0440\\u0430\\u0431\\u043e\\u0447\\u0435\\u0433\\u043e \\u043c\\u0435\\u0441\\u0442\\u0430, \\u0438\\u043b\\u0438 \\u0432\\u043d\\u0435 \\u0434\\u043e\\u0441\\u0442\\u0443\\u043f\\u0430 \\u0440\\u043e\\u0437\\u0435\\u0442\\u043a\\u0438, \\u0432\\u043e \\u0432\\u0441\\u0435\\u0445 \\u043e\\u0441\\u0442\\u0430\\u043b\\u044c\\u043d\\u044b\\u043c \\u0441\\u043b\\u0443\\u0447\\u0430\\u044f\\u0445 \\u043d\\u0443\\u0436\\u043d\\u043e \\u043f\\u043e\\u043a\\u0443\\u043f\\u0430\\u0442\\u044c \\u0441\\u0442\\u0430\\u0446\\u0438\\u043e\\u043d\\u0430\\u0440\\u043d\\u044b\\u0439 \\u041f\\u041a\&q;,\&q;dignity\&q;:\&q;\&q;,\&q;shortcomings\&q;:\&q;\&q;,\&q;positive\&q;:\&q;0\&q;,\&q;negative\&q;:\&q;0\&q;,\&q;replies\&q;:[],\&q;marks\&q;:[],\&q;questions\&q;:[],\&q;created_iso\&q;:\&q;2020-03-30T18:43:34+03:00\&q;}&q;,&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;}],&q;mark_percent&q;:&q;60%&q;,&q;votes&q;:{&q;positive&q;:2,&q;negative&q;:3,&q;total&q;:5}},{&q;id&q;:45596840,&q;usertitle&q;:&q;Вова Стефюк&q;,&q;mark&q;:5,&q;percent_dignity&q;:&q;0&q;,&q;text&q;:&q;Відмінний ноут за цю ціну)&q;,&q;dignity&q;:&q;Ціна)&q;,&q;shortcomings&q;:&q;Немає як за цю ціну&q;,&q;from_buyer&q;:false,&q;created&q;:{&q;pop_date&q;:&q;29 февраля 2020&q;,&q;year&q;:&q;2020&q;,&q;month&q;:2,&q;day&q;:&q;29&q;},&q;questions&q;:{&q;combobox&q;:[],&q;marks&q;:[],&q;integer&q;:[]},&q;attachments&q;:[],&q;lightboxattachmentsjson&q;:&q;[]&q;,&q;commentdatajson&q;:{&q;id&q;:45596840,&q;goods_id&q;:&q;181862304&q;,&q;user_title&q;:&q;Вова Стефюк&q;,&q;created&q;:&q;29 февраля 2020&q;,&q;from_buyer&q;:false,&q;mark&q;:5,&q;text&q;:&q;Відмінний ноут за цю ціну)&q;,&q;dignity&q;:&q;Ціна)&q;,&q;shortcomings&q;:&q;Немає як за цю ціну&q;,&q;positive&q;:&q;0&q;,&q;negative&q;:&q;0&q;,&q;replies&q;:[],&q;marks&q;:[],&q;questions&q;:[],&q;created_iso&q;:&q;2020-02-29T13:27:53+02:00&q;},&q;is_vendor_comment&q;:false,&q;vendor_logo&q;:&q;&q;,&q;seller_id&q;:5,&q;replies&q;:[],&q;mark_percent&q;:&q;100%&q;,&q;votes&q;:{&q;positive&q;:0,&q;negative&q;:0,&q;total&q;:0}}],&q;most_useful&q;:[],&q;questions&q;:{&q;marks&q;:[],&q;combobox&q;:[]},&q;form_focus&q;:false,&q;complaints_reasons&q;:[{&q;title&q;:&q;нецензурная лексика, оскорбление или агрессия&q;,&q;id&q;:1},{&q;title&q;:&q;явная реклама (спам)&q;,&q;id&q;:3},{&q;title&q;:&q;ссылки на незаконные сайты и программы&q;,&q;id&q;:2},{&q;title&q;:&q;не относится к обсуждаемой теме&q;,&q;id&q;:4},{&q;title&q;:&q;другая причина&q;,&q;id&q;:5}],&q;gallery_data&q;:{&q;attachments&q;:[],&q;comments&q;:[]},&q;active_sort&q;:{&q;title&q;:&q;По дате&q;,&q;is_selected&q;:true,&q;name&q;:&q;date&q;,&q;href&q;:&q;/&q;}}},&q;headers&q;:{&q;server&q;:[&q;nginx/1.16.1&q;],&q;date&q;:[&q;Wed, 08 Apr 2020 20:33:37 GMT&q;],&q;content-type&q;:[&q;application/json; charset=UTF-8&q;],&q;transfer-encoding&q;:[&q;chunked&q;],&q;connection&q;:[&q;keep-alive&q;],&q;keep-alive&q;:[&q;timeout=20&q;],&q;vary&q;:[&q;Accept-Encoding&q;],&q;domain&q;:[&q;product-api, product-api&q;]},&q;status&q;:200,&q;statusText&q;:&q;OK&q;,&q;url&q;:&q;https://product-api.rozetka.com.ua/v3/comments/get?front-type=xl&a;goods=181862304&a;page=1&a;sort=date&a;limit=10&a;lang=ru&q;}}"

In [17]:
s.find('Алексей')

78841

In [20]:
s.find('mark&q;:', 78841)

78855

In [24]:
s[78855+len('mark&q;:')]

'4'

In [26]:
s[78810:78900]

'q;:45842363,&q;usertitle&q;:&q;Алексей&q;,&q;mark&q;:4,&q;percent_dignity&q;:&q;100&q;,&q;'

In [223]:
url = "https://rozetka.com.ua/ua/notebooks/c80004/page=2/"

In [224]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'html.parser')

In [225]:
soup

<!DOCTYPE html>
<html lang="ru"><head>
<title>Ноутбуки Харків. Купити ноутбук в Харкові, відгуки, продаж | ROZETKA</title><link href="https://rozetka.md/ua/notebooks/c80004/page=2/" hreflang="ru-ro" id="alternate_ru-ro" rel="alternate"/><link href="https://rozetka.md/ua/notebooks/c80004/page=2/" hreflang="ru-md" id="alternate_ru-md" rel="alternate"/>
<meta content="Купити ноутбук в Харкові в інтернет магазині ⏩ ROZETKA. Висока якість, ✍️️ відгуки, ⚡ швидка доставка по всій Україні, ✨ 【ГАРАНТІЯ】!" name="description"/>
<meta content="ноутбуки Харків, купити ноутбук в Харкові" name="keywords"/>
<meta content="noindex,nofollow" name="robots"/>
<meta charset="utf-8"/>
<script> dataLayer = []; </script>
<script>
        const Cart = new Object({
            purchases   : [],
            setPurchases: function (purchases) {
                this.purchases = purchases;
            },
            getPurchases: function () {
                return this.purchases;
            },
        });

    <

In [226]:
script_text = soup.find('script', id='rz-client-state').text
script_text

'{&q;G.https://xl-catalog-api.rozetka.com.ua/v2/super-portals/getList?from_node=1&q;:{&q;body&q;:{&q;data&q;:[4627949,80253,2394287,4627893,2394297,88468,368610,4627858,80261,4630262,4629305,80025,1162030,2337947,2337942,2028477,4626923,4628418,4429255,4659999,4625535,4627851,4625734,84625]},&q;headers&q;:{&q;server&q;:[&q;nginx/1.16.1&q;],&q;date&q;:[&q;Thu, 09 Apr 2020 01:36:32 GMT&q;],&q;content-type&q;:[&q;application/json; charset=UTF-8&q;],&q;transfer-encoding&q;:[&q;chunked&q;],&q;connection&q;:[&q;keep-alive&q;],&q;keep-alive&q;:[&q;timeout=20&q;],&q;vary&q;:[&q;Accept-Encoding&q;],&q;x-powered-by&q;:[&q;PHP/7.1.33-1+0~20191027.27+debian9~1.gbpd10139&q;],&q;access-control-expose-headers&q;:[&q;X-Pagination-Total-Count, X-Pagination-Page-Count, X-Pagination-Current-Page, X-Pagination-Per-Page&q;],&q;domain&q;:[&q;catalog-api, catalog-api&q;]},&q;status&q;:200,&q;statusText&q;:&q;OK&q;,&q;url&q;:&q;https://xl-catalog-api.rozetka.com.ua/v2/super-portals/getList?from_node=1&q;},&q;

In [227]:
import re

In [228]:
urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', script_text)

In [229]:
urls

['https://xl-catalog-api.rozetka.com.ua/v2/super-portals/getList?from_node=1&q;:',
 'https://xl-catalog-api.rozetka.com.ua/v2/super-portals/getList?from_node=1&q;',
 'https://xl-static.rozetka.com.ua/assets/i18n/ru.json?&q;:',
 'https://xl-static.rozetka.com.ua/assets/i18n/ru.json&q;',
 'https://xl-static.rozetka.com.ua/assets/i18n/ua.json?&q;:',
 'https://xl-static.rozetka.com.ua/assets/i18n/ua.json&q;',
 'https://common-api.rozetka.com.ua/v2/fat-menu/basic?front-type=xl&a;lang=ua?&q;:',
 'https://rozetka.com.ua/computers-notebooks/c80253/&q;,&q;outer_link&q;:false',
 'https://rozetka.com.ua/telefony-tv-i-ehlektronika/c4627949/&q;,&q;outer_link&q;:false',
 'https://bt.rozetka.com.ua/&q;,&q;outer_link&q;:false',
 'https://rozetka.com.ua/tovary-dlya-doma/c2394287/&q;,&q;outer_link&q;:false',
 'https://rozetka.com.ua/instrumenty-i-avtotovary/c4627858/&q;,&q;outer_link&q;:false',
 'https://rozetka.com.ua/santekhnika-i-remont/c4628418/&q;,&q;outer_link&q;:false',
 'https://rozetka.com.ua/d

In [230]:
urls[0].startswith('https://xl-catalog-api')

True

In [231]:
urls

['https://xl-catalog-api.rozetka.com.ua/v2/super-portals/getList?from_node=1&q;:',
 'https://xl-catalog-api.rozetka.com.ua/v2/super-portals/getList?from_node=1&q;',
 'https://xl-static.rozetka.com.ua/assets/i18n/ru.json?&q;:',
 'https://xl-static.rozetka.com.ua/assets/i18n/ru.json&q;',
 'https://xl-static.rozetka.com.ua/assets/i18n/ua.json?&q;:',
 'https://xl-static.rozetka.com.ua/assets/i18n/ua.json&q;',
 'https://common-api.rozetka.com.ua/v2/fat-menu/basic?front-type=xl&a;lang=ua?&q;:',
 'https://rozetka.com.ua/computers-notebooks/c80253/&q;,&q;outer_link&q;:false',
 'https://rozetka.com.ua/telefony-tv-i-ehlektronika/c4627949/&q;,&q;outer_link&q;:false',
 'https://bt.rozetka.com.ua/&q;,&q;outer_link&q;:false',
 'https://rozetka.com.ua/tovary-dlya-doma/c2394287/&q;,&q;outer_link&q;:false',
 'https://rozetka.com.ua/instrumenty-i-avtotovary/c4627858/&q;,&q;outer_link&q;:false',
 'https://rozetka.com.ua/santekhnika-i-remont/c4628418/&q;,&q;outer_link&q;:false',
 'https://rozetka.com.ua/d

In [232]:
urls.count('https://rozetka.com.ua/ua/dell_i35c445dil_73b/p181862304/')

0

In [233]:
filtered_url = [url for url in urls if 'https://rozetka.com.ua' in url and 'comments_amount&q' in url]

In [234]:
len(filtered_url)

35

In [235]:
filtered_url

['https://rozetka.com.ua/ua/hp_pavilion_gaming_15_8kq04ea/p136181681/&q;,&q;comments_amount&q;:15,&q;sell_status&q;:&q;out_of_stock&q;,&q;comments_mark&q;:5,&q;category_id&q;:80004,&q;seller_id&q;:5,&q;merchant_id&q;:1,&q;brand&q;:&q;HP&q;,&q;brand_id&q;:5,&q;group_id&q;:0,&q;group_name&q;:null,&q;group_title&q;:null,&q;pl_bonus_charge_pcs&q;:0,&q;pl_use_instant_bonus&q;:0,&q;state&q;:&q;new&q;,&q;docket&q;:&q;',
 'https://rozetka.com.ua/ua/dell_n5104vn5590emea01_2005_ubu_rail_08/p150765329/&q;,&q;comments_amount&q;:14,&q;sell_status&q;:&q;available&q;,&q;comments_mark&q;:4,&q;category_id&q;:80004,&q;seller_id&q;:5,&q;merchant_id&q;:1,&q;brand&q;:&q;Dell&q;,&q;brand_id&q;:7,&q;group_id&q;:0,&q;group_name&q;:null,&q;group_title&q;:null,&q;pl_bonus_charge_pcs&q;:0,&q;pl_use_instant_bonus&q;:0,&q;state&q;:&q;new&q;,&q;docket&q;:&q;',
 'https://rozetka.com.ua/ua/dell_i3558s3niw_75b/p182669383/&q;,&q;comments_amount&q;:1,&q;sell_status&q;:&q;available&q;,&q;comments_mark&q;:4,&q;category_id

In [11]:
import re

In [12]:
get_comment_url(filtered_url[0])

NameError: name 'get_comment_url' is not defined

Get notebooks comments urls

In [28]:
def get_comment_url_by_text(text_with_url: str):
    end_url_pos = text_with_url.find('&q;')
    url = text_with_url[:end_url_pos]
    comment_amount_tag = 'comments_amount&q;:'
    comment_pos = text_with_url.find('comment_amount_tag')
    num_comments = text_with_url[comment_pos + len(comment_amount_tag)]
    if num_comments == '0':
        return None
    return f"{url}comments/"

In [29]:
def get_comment_urls(page_url):
    result = requests.get(page_url)
    soup = BeautifulSoup(result.text, 'html.parser')
    script_text = soup.find('script', id='rz-client-state').text
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', script_text)
    filtered_urls = [url for url in urls if 'https://rozetka.com.ua' in url and 'comments_amount&q' in url]
    comment_urls = [ get_comment_url_by_text(filtered_url) for filtered_url in filtered_urls]
    return comment_urls

In [30]:
len(get_comment_urls('https://rozetka.com.ua/ua/notebooks/c80004/page=1/'))

35

In [31]:
notebooks_url = [f"https://rozetka.com.ua/ua/notebooks/c80004/page={page}/" for page in range(1, 35)]

In [32]:
comment_url_list = []
for notebook_url in notebooks_url:
    comment_urls = get_comment_urls(notebook_url)
    comment_url_list += comment_urls

In [33]:
len(comment_url_list)

883

In [38]:
def get_all_comments(comment_url_list):
        full_result = {
        'title' : [],
        'prons' : [],
        'cons' : [],
        'mark' : []
    }
        for comment_url in comment_url_list:
            parser = UserReviewParser(comment_url)
            result = parser.parse_review_data()
            full_result['title'] += result['title']
            full_result['prons'] += result['prons']
            full_result['cons'] += result['cons']
            full_result['mark'] += result['mark']
    
        return full_result

In [39]:
len(comment_url_list)

883

In [40]:
%time full_result = get_all_comments(comment_url_list)

https://rozetka.com.ua/ua/msi_a10m_653xua/p138041235/comments/
https://rozetka.com.ua/ua/lenovo_81mx0034ra/p172824226/comments/
https://rozetka.com.ua/ua/dell_i3558s3nil_75b/p191968435/comments/
https://rozetka.com.ua/ua/hp_pavilion_gaming_15_8ks39ea/p136168934/comments/
https://rozetka.com.ua/ua/acer_nx_hf8eu_010/p132336040/comments/
https://rozetka.com.ua/ua/hp_6mq34ea/p117234193/comments/
https://rozetka.com.ua/ua/hp_8ac84ea/p172084459/comments/
https://rozetka.com.ua/ua/acer_nx_gzgeu_004/p44415672/comments/
https://rozetka.com.ua/ua/hp_6mq29ea/p88416435/comments/
https://rozetka.com.ua/ua/hp_8ab66es/p148031073/comments/
https://rozetka.com.ua/ua/asus_90nx0242_m00870/p109658934/comments/
https://rozetka.com.ua/ua/lenovo_81sx00hhra/p122449278/comments/
https://rozetka.com.ua/ua/lenovo_81sx00hhra/p122449278/comments/
https://rozetka.com.ua/ua/lenovo_81nh0050ra/p104573514/comments/
https://rozetka.com.ua/ua/acer_nx_hgaeu_005/p138690540/comments/
https://rozetka.com.ua/ua/hp_omen_x_2s_7

In [41]:
len(full_result['title'])

385

In [ ]:
full_result['title']

In [287]:
43 * 879 / 20 

1889.85

In [4]:
len(full_result['prons'])

NameError: name 'full_result' is not defined

In [42]:
pd.DataFrame(full_result)

,title,prons,cons,mark
0,Тут type C thunderbolt или обычный ?,None,None,4
1,Как оформить кредит на этот ноут,None,None,5
2,Через полтора месяца накрылся кулер ...,"Подсветка, начинка, ips","Кулер слабый, сильно греется; интернет разъем ...",3
3,"Соглашусь, что для своей ценовой категории, мо...","низкая цена, достаточно стильный дизайн",качество экрана,3
4,Купил для Мамы что б можно было связаться с ро...,Цена и качество,"нет проводного инета (решается переходником, д...",5
...,...,...,...,...
380,Добрый день! Меня интересует вопрос по поводу ...,None,None,4
381,Добрий день. Ви можете сказати яка планка памя...,None,None,5
382,Отличный ноут за свои деньги. Хорошо сбалансир...,Мощная двухкулерная система охлаждения,Долбаное ПО для настройки подсветки,5
383,Отличный рабочий ноутбук.,"Добротный корпус, IPS-экран, возможность устан...",Нет,5
